In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/MS/ETAnalysis/Data/Tremor_eval_dl.zip" -d "./data"

# Setting

In [ ]:
import math, collections
from pathlib import Path
from typing import *
import os, re, math, json, gc, random, time, glob, warnings, itertools
import warnings
from tqdm import tqdm
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

from dataclasses import dataclass
from functools import lru_cache

from scipy.signal import butter, filtfilt
from scipy.ndimage import median_filter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
_SPLIT_FILE_RE = re.compile(r".*_\d+_[123]$")  # 예: .../UniV_m_rawdata_0_1.csv (이미 분할된 파일 패턴)

def split_and_delete_multidirect_fs(
    root: Path = Path("/content/data"),
    *,
    c1_col: str = "c1",
    median_size: int = 5,
    suffix_map: Dict[int, str] = None,   # {100:"_1", 200:"_2", 300:"_3"}
    overwrite: bool = True
) -> dict:
    """
    /content/data 하위에서 디렉토리명이 'MultiDirect'(대소문자 무시)인 모든 CSV를 찾아:
      1) c1 전처리 (round → abs → median_filter(size=median_size) → ×100)
      2) c1=={100,200,300} 행을 각각 _1/_2/_3 접미사로 새 CSV 저장
      3) 분할본이 하나라도 생성되면 원본 CSV 삭제

    이미 분할된 파일(…_0_1.csv 등, 끝에 '_<trial>_<1|2|3>.csv')은 재처리/삭제 대상에서 제외.

    Returns
    -------
    {'saved': [...], 'deleted': [...], 'kept': [...], 'errors': [...]}
    """
    if suffix_map is None:
        suffix_map = {100: "_1", 200: "_2", 300: "_3"}

    # MultiDirect 디렉토리 하위의 모든 CSV 수집 (대소문자 무시)
    csv_files: List[Path] = [
        p for p in root.rglob("*.csv")
        if any(part.lower() == "multidirect" for part in p.parts)
    ]

    saved, deleted, kept, errors = [], [], [], []
    seen = set()

    for p in sorted(csv_files):
        if p in seen:
            continue
        seen.add(p)

        # 이미 분할된 파일 패턴이면 스킵 (…_trialLevel_c1Level.csv)
        if _SPLIT_FILE_RE.match(p.stem):
            kept.append(str(p))  # 그대로 유지
            continue

        # CSV 읽기
        try:
            df = pd.read_csv(p)
        except Exception as e:
            errors.append(f"[READ-ERROR] {p}: {e}")
            kept.append(str(p))
            continue

        if c1_col not in df.columns:
            errors.append(f"[MISSING-COLUMN] {p}: '{c1_col}' not found")
            kept.append(str(p))
            continue

        # c1 전처리: round → abs → median filter → ×100
        try:
            c1_raw = pd.to_numeric(df[c1_col], errors="coerce").to_numpy(dtype=float)
            c1_int = np.rint(c1_raw).astype(np.int64)
            c1_abs = np.abs(c1_int)
            c1_med = median_filter(c1_abs, size=median_size, mode="nearest").astype(np.int64)
            c1 = c1_med * 100  # 기대: {100,200,300}
        except Exception as e:
            errors.append(f"[C1-PROCESS-ERROR] {p}: {e}")
            kept.append(str(p))
            continue

        # 분할 저장
        wrote_any = False
        for lv, suf in suffix_map.items():
            m = (c1 == lv)
            if np.any(m):
                sub_df = df.loc[m].reset_index(drop=True)
                out_path = p.with_name(f"{p.stem}{suf}{p.suffix}")
                try:
                    if (not overwrite) and out_path.exists():
                        # 덮어쓰기 금지 시 존재하면 스킵
                        pass
                    else:
                        sub_df.to_csv(out_path, index=False)
                        saved.append(str(out_path))
                        wrote_any = True
                except Exception as e:
                    errors.append(f"[WRITE-ERROR] {out_path}: {e}")

        # 분할본이 하나라도 생겼다면 원본 삭제
        if wrote_any:
            try:
                p.unlink()
                deleted.append(str(p))
            except Exception as e:
                kept.append(str(p))
                errors.append(f"[DELETE-ERROR] {p}: {e}")
        else:
            kept.append(str(p))

    return {"saved": saved, "deleted": deleted, "kept": kept, "errors": errors}

split_and_delete_multidirect_fs(Path("/content/data"))

{'saved': ['/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_0_1.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_0_2.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_0_3.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_1_1.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_1_2.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_1_3.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_3_1.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_3_2.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_3_3.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_4_1.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_4_2.csv',
  '/content/data/ET01_1_01232017/MultiDirect/UniV_m_rawdata_4_3.csv',
  '/content/data/ET01_3_02062017/MultiDirect/UniV_m_rawdata_0_1.csv',
  '/content/data/ET01_3_02062017/MultiDirect/UniV_m_rawdata_0_2.csv',
  '/content

In [ ]:
ROOT = Path("/content/data")
OUT_DIR = Path("/content")
LABEL_DIR = Path('/content/drive/MyDrive/MS/ETAnalysis/Data/relabel_md_k5.csv')

SEED = 42
FS      = 100.0
SEG_LEN = 512
HOP     = 512
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH = 128
NUM_WORKERS = 4 if DEVICE == "cuda" else 2
USECOLS = ["accel_x","accel_y","accel_z","coor_x","coor_y"]
TARGET = 'target_k5'

_DIR_RE   = re.compile(r"ET(\d{2})_(\d)_(\d{8})")
_TASK_RE  = re.compile(r"(Spiral|Maze|MultiDirect)", re.IGNORECASE)

TASK2IDX = {"spiral": 0, "maze": 1, "multidirect": 2, "unknown": 3}


# ── 재현성 설정 ───────────────────────────────────────────
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Model

## LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, in_c, hidden_size, num_layers, dropout=0.1):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=in_c,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
        )

    def forward(self, x):
        h_n, c_n = self.lstm(x)
        return h_n

    def encode(self, x):
        return self.forward(x)

## ResNet18

In [ ]:
class BasicBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(BasicBlock1D, self).__init__()
        # 수정사항 1. padding 자동 계산
        padding = (kernel_size - 1) // 2

        self.conv1 = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=False
        )
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=1,
            padding=padding,
            bias=False
        )
        self.bn2 = nn.BatchNorm1d(out_channels)
        # shortcut
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

        # 수정사항 2. ReLU 모듈화
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        out_main = self.conv1(x)
        out_main = self.bn1(out_main)
        out_main = self.relu(out_main)
        out_main = self.conv2(out_main)
        out_main = self.bn2(out_main)

        out_sc = self.shortcut(x)

        y = self.relu(out_main + out_sc)
        return y


class ResNet18_1D(nn.Module):
    def __init__(self, num_classes, input_channels, d_model):
        super(ResNet18_1D, self).__init__()

        self.stem = nn.Sequential(
            nn.Conv1d(
                in_channels=input_channels,
                out_channels=64,
                kernel_size=7,
                stride=2,
                padding=3,
                bias=False
            ),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        )
        self.stage1 = self._make_stage(64, 64, [1, 1])
        self.stage2 = self._make_stage(64, 128, [2, 1])
        self.stage3 = self._make_stage(128, 256, [2, 1])
        self.stage4 = self._make_stage(256, 512, [2, 1])

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.Flatten(),
            nn.Linear(in_features=512, out_features=num_classes)
        )

        self.proj = nn.Conv1d(
            in_channels=512,
            out_channels=d_model,
            kernel_size=1,
            stride=1,
            bias=False,
        )
        self.apply(self._initialize_weights)

    def _make_stage(self, in_channels, out_channels, strides: list):
        layers = []
        for s in strides:
            layers.append(
                BasicBlock1D(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=3,
                    stride=s
                )
            )
            in_channels = out_channels
        return nn.Sequential(*layers)

    def _initialize_weights(self, m):
        if isinstance(m, nn.Conv1d):
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))
            nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.stem(x)

        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        x = self.head(x)
        return x

    def encode(self, x):
        x = x.permute(0, 2, 1)
        x = self.stem(x)

        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.proj(x)

        return x.permute(0, 2, 1)

## TimesNet

In [ ]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]


class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super().__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, dropout=0.1):
        super().__init__()

        self.value_embedding = TokenEmbedding(c_in=c_in, d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.value_embedding(x) + self.position_embedding(x)
        return self.dropout(x)


class Inception_Block_V1(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=6, init_weight=True):
        super(Inception_Block_V1, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=2 * i + 1, padding=i))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res

class Inception_Block_V2(nn.Module):
    def __init__(self, in_channels, out_channels, num_kernels=7, init_weight=True):
        super(Inception_Block_V2, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_kernels = num_kernels
        kernels = []
        for i in range(self.num_kernels // 2):
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=[1, 2 * i + 3], padding=[0, i + 1]))
            kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=[2 * i + 3, 1], padding=[i + 1, 0]))
        kernels.append(nn.Conv2d(in_channels, out_channels, kernel_size=1))
        self.kernels = nn.ModuleList(kernels)
        if init_weight:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        res_list = []
        for i in range(self.num_kernels // 2 * 2 + 1):
            res_list.append(self.kernels[i](x))
        res = torch.stack(res_list, dim=-1).mean(-1)
        return res


def FFT_for_Period(x, k=2, fs=100.0, f_min=None):
    B, T, C = x.shape
    xf = torch.fft.rfft(x, dim=1)
    frequency_list = abs(xf).mean(0).mean(-1)

    if f_min is None:
        frequency_list[0] = 0
    else:
        # Zero-out all components ≤ f_min (including DC)
        idx_cut = int(math.floor(float(f_min) * T / float(fs)))   # bin index for f_min
        frequency_list[:idx_cut + 1] = 0

    _, top_list = torch.topk(frequency_list, k)
    top_list = top_list.detach().cpu().numpy()
    period = x.shape[1] // top_list
    return period, abs(xf).mean(-1)[:, top_list]


class TimesBlock(nn.Module):
    def __init__(self, seq_len, d_model, d_ff, top_k, f_min=None):
        super().__init__()
        self.seq_len = seq_len
        self.k = top_k
        self.f_min = f_min

        self.conv = nn.Sequential(
            Inception_Block_V2(d_model, d_ff),
            nn.GELU(),
            Inception_Block_V2(d_ff, d_model),
        )

    def forward(self, x):
        B, T, N = x.size()
        # period_list, period_weight = FFT_for_Period(x, k=self.k, f_min=self.f_min)
        period_list, period_weight = FFT_for_Period(x, k=self.k)

        res = []
        for i in range(self.k):
            period = period_list[i]
            # padding
            if self.seq_len % period != 0:
                length = ((self.seq_len // period) + 1) * period
                padding = torch.zeros([x.shape[0], (length - self.seq_len), x.shape[2]]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = self.seq_len
                out = x
            # reshape
            out = out.reshape(B, length // period, period, N).permute(0, 3, 1, 2).contiguous()
            # 2D conv
            out = self.conv(out)
            # reshape back
            out = out.permute(0, 2, 3, 1).reshape(B, -1 ,N)
            res.append(out[:, :self.seq_len, :])
        res = torch.stack(res, dim=-1) # B T N k
        # adaptive aggregation
        period_weight = F.softmax(period_weight, dim=-1)
        period_weight = period_weight.unsqueeze(
            1).unsqueeze(1).repeat(1, T, N, 1)
        res = torch.sum(res * period_weight, -1)
        # residual connection
        res = res + x
        return res


class TimesNet(nn.Module):
    def __init__(self, num_class, seq_len, enc_in, d_model, d_ff, e_layers, top_k, f_min=None, dropout=0.1):
        super().__init__()
        self.enc_embedding = DataEmbedding(enc_in, d_model, dropout)
        self.model = nn.ModuleList([TimesBlock(seq_len, d_model, d_ff, top_k, f_min)
                                    for _ in range(e_layers)])
        self.layer = e_layers
        self.layer_norm = nn.LayerNorm(d_model)
        self.act = F.gelu
        self.dropout = nn.Dropout(dropout)
        self.projection = nn.Linear(d_model * seq_len, num_class)
        self.d_model = d_model
        self.seq_len = seq_len
        self.num_class = num_class

    # ===== 추가: 세그먼트 임베딩만 뽑는 메서드 =====
    def encode(self, x_enc):  # x_enc: [B, T, C]
        enc_out = self.enc_embedding(x_enc)  # [B,T,d_model]
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        # y = self.act(enc_out)
        # y = self.dropout(y)
        # # 간단하고 안정적인 통계 풀링(평균). 필요하면 mean+std concat으로 확장 가능.
        # feat = y.mean(dim=1)                  # [B, d_model]
        return enc_out

    # 기존 세그먼트 단일 분류도 유지
    def classification(self, x):
        enc_out = self.enc_embedding(x)
        for i in range(self.layer):
            enc_out = self.layer_norm(self.model[i](enc_out))
        output = self.act(enc_out)
        output = self.dropout(output)
        output = output.reshape(output.shape[0], -1)
        return self.projection(output)

    def forward(self, x):
        return self.classification(x)  # [B, num_class]

## MyWaveNet

In [ ]:
class AttnPoolHead(nn.Module):
    def __init__(self, d_model, num_class, p=0.1):
        super().__init__()
        self.score = nn.Linear(d_model, 1, bias=False)
        self.proj = nn.Sequential(
            nn.GELU(),
            nn.Dropout(p),
            nn.Linear(d_model, num_class)
        )

    def forward(self, x):
        # x = B T D
        s = self.score(x).squeeze(-1) # B T
        w = F.softmax(s, dim=1) # B T
        z = torch.einsum('bt,btd->bd', w, x) # B d
        return self.proj(z)

class _GatedResidualUnit(nn.Module):
    def __init__(self, res_ch, skip_ch, dilation):
        super().__init__()
        self.conv_filter = nn.Conv1d(res_ch, res_ch, 2,
                                     padding=dilation, dilation=dilation)
        self.conv_gate   = nn.Conv1d(res_ch, res_ch, 2,
                                     padding=dilation, dilation=dilation)
        self.conv_res    = nn.Conv1d(res_ch, res_ch, 1)
        self.conv_skip   = nn.Conv1d(res_ch, skip_ch, 1)

    def forward(self, x):
        # causal padding 보정 (Conv1d 는 causal 지원 X → 수동 crop)
        out_f = self.conv_filter(x)[:, :, :-self.conv_filter.dilation[0]]
        out_g = self.conv_gate  (x)[:, :, :-self.conv_gate.dilation[0]]
        z = torch.tanh(out_f) * torch.sigmoid(out_g)          # gated
        res = self.conv_res(z)
        skip= self.conv_skip(z)
        return (x + res), skip


class MyWaveNet(nn.Module):
    def __init__(
            self,
            in_ch: int,
            res_ch: int,
            skip_ch: int,
            dilations: list,
            n_classes: int = 4,
            n_stacks: int = 2,
    ):
        super().__init__()

        self.skip_ch = skip_ch

        self.ln_pre_cls = nn.LayerNorm(skip_ch)

        self.input_proj = nn.Conv1d(in_ch, res_ch, kernel_size=1)

        def _stack_blocks(nstacks: int, dilations_list, block_cls, res_ch, skip_ch):
            blocks = []
            for _ in range(nstacks):
                for d in dilations_list:
                    blocks.append(block_cls(res_ch, skip_ch, d))
            return nn.ModuleList(blocks)

        self.res_blocks = _stack_blocks(n_stacks, dilations, _GatedResidualUnit, res_ch, skip_ch)

        self.cls_head = AttnPoolHead(skip_ch, n_classes)

    def encode(self, x):
        x = x.permute(0, 2, 1) # B T C -> B C T
        x = self.input_proj(x)

        skip_sum = None
        for blk in self.res_blocks:
            x, skip = blk(x)
            skip_sum = skip if skip_sum is None else skip_sum + skip

        return skip_sum.permute(0, 2, 1) # B T C

    def forward(self, x):
        # x : B C T
        skip_sum = self.encode(x)

        skip_sum = self.ln_pre_cls(skip_sum) # B T C

        logits_global = self.cls_head(skip_sum)             # (B, n_classes)

        return logits_global

## MIL_MultiModal

In [ ]:
class GAPTimeReadout(nn.Module):
    """
    x: [B*, T, D] -> z: [B*, D]  (시간축 평균)
    """
    def __init__(self, p: float = 0.0):
        super().__init__()
        self.drop = nn.Dropout(p)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        z = x.mean(dim=1)  # [B*, D]
        return self.drop(z)

class AttnTimeReadout(nn.Module):
    """
    x: [B*, T, D] → (z: [B*, D], w_time: [B*, T])
    """
    def __init__(self, d_model: int, p: float = 0.1):
        super().__init__()
        self.score = nn.Linear(d_model, 1, bias=False)
        self.drop = nn.Dropout(p)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        s = self.score(x).squeeze(-1)          # [B*, T]
        w = F.softmax(s, dim=1)                # [B*, T]
        z = torch.einsum('bt,btd->bd', w, x)   # [B*, D]
        return self.drop(z), w

# ─────────────────────────────────────────────
# 인스턴스 수준 → bag 수준으로 집계하는 Gated Attention MIL
# (Ilse et al., 2018 스타일)
# ─────────────────────────────────────────────
class AttnMILHead(nn.Module):
    """
    feats: [B, N, D], mask: [B, N] (True=valid)
    반환: logits: [B, C], alpha_inst: [B, N]
    """
    def __init__(self, feat_dim: int, num_classes: int, attn_dim: int = 128, dropout: float = 0.1):
        super().__init__()
        self.V = nn.Linear(feat_dim, attn_dim)
        self.U = nn.Linear(feat_dim, attn_dim)
        self.w = nn.Linear(attn_dim, 1)
        self.drop = nn.Dropout(dropout)
        self.classifier = nn.Linear(feat_dim, num_classes)

    def forward(self, feats: torch.Tensor, mask: Optional[torch.Tensor] = None):
        B, N, D = feats.shape
        Vh = torch.tanh(self.V(feats))               # [B,N,A]
        Uh = torch.sigmoid(self.U(feats))            # [B,N,A]
        A  = self.w(self.drop(Vh * Uh)).squeeze(-1)  # [B,N]
        if mask is not None:
            A = A.masked_fill(~mask, float('-inf'))
        alpha = torch.softmax(A, dim=1)              # [B,N]
        z = torch.einsum('bn,bnd->bd', alpha, feats) # [B,D]
        logits = self.classifier(self.drop(z))       # [B,C]
        return logits, alpha


class MIL_MultiModalTimesNet(nn.Module):
    """
    입력:
      x_acc_bag  : [B, N, T, C_acc]
      x_traj_bag : [B, N, T, C_traj]
      mask       : [B, N] (True=valid)

    출력:
      sev_logits : [B, num_sclass]
      task_logits: [B, num_tclass]
      aux        : dict (해석용 가중치들)
    """
    def __init__(
        self,
        acc_encoder, traj_encoder,
        d_model: int,
        num_sclass: int,
        *,
        ma_heads: int = 2,
        ma_dropout: float = 0.1,
        time_dropout: float = 0.1,
        mil_attn_dim: int = 64,
        time_pool : str = 'attn',
    ):
        super().__init__()
        self.acc_encoder = acc_encoder
        self.traj_encoder = traj_encoder

        self.acc_ln  = nn.LayerNorm(d_model)
        self.traj_ln = nn.LayerNorm(d_model)

        self.ma = nn.MultiheadAttention(
            embed_dim=d_model, num_heads=ma_heads, dropout=ma_dropout, batch_first=True
        )

        self.time_pool = time_pool
        if self.time_pool == "attn":
        # 시간축 readout (인스턴스 임베딩 만들기)
            self.time_readout_sev  = AttnTimeReadout(d_model, p=time_dropout)  # attn_output → z_sev_inst
        elif self.time_pool == 'gap':
            self.time_readout_sev = GAPTimeReadout(p=time_dropout)
        # 인스턴스→bag MIL (집계 후 최종 로짓)
        self.mil_sev  = AttnMILHead(feat_dim=d_model, num_classes=num_sclass, attn_dim=mil_attn_dim, dropout=ma_dropout)

    @torch.no_grad()
    def _check_shapes(self, x_acc_bag, x_traj_bag, mask):
        assert x_acc_bag.dim() == 4 and x_traj_bag.dim() == 4, "x_* shape must be [B,N,T,C]"
        assert x_acc_bag.shape[:3] == x_traj_bag.shape[:3], "ACC/TRAJ must share [B,N,T]"
        if mask is not None:
            assert mask.shape == x_acc_bag.shape[:2], "mask shape must be [B,N]"

    def forward(self, x_acc_bag: torch.Tensor, x_traj_bag: torch.Tensor, mask: Optional[torch.Tensor] = None):
        self._check_shapes(x_acc_bag, x_traj_bag, mask)
        B, N, T, C_acc = x_acc_bag.shape
        _, _, _, C_traj = x_traj_bag.shape

        # ── (1) 인스턴스 펼치기 ─────────────────────────────────────
        x_acc  = x_acc_bag.reshape(B*N, T, C_acc)      # [B*N, T, C_acc]
        x_traj = x_traj_bag.reshape(B*N, T, C_traj)    # [B*N, T, C_traj]

        # ── (2) 인코더: 시계열 → 시퀀스 임베딩 ─────────────────────
        y_acc  = self.acc_encoder.encode(x_acc)          # [B*N, T, D]
        y_traj = self.traj_encoder.encode(x_traj)      # [B*N, T, D]

        y_acc = self.acc_ln(y_acc)
        y_traj = self.traj_ln(y_traj)

        # ── (3) Cross-Attention: traj(query) ← acc(key/value) ─────
        attn_output, attn_w = self.ma(y_traj, y_acc, y_acc)  # [B*N, T, D], [B*N, num_heads, T, T] (batch_first=True)

        # ── (4) 시간축 어텐션 풀링(인스턴스 임베딩) ─────────────────
        if self.time_pool == "attn":
            z_sev_inst,   w_time_sev  = self.time_readout_sev(attn_output) # [B*N, D], [B*N, T]
        elif self.time_pool == 'gap':
            z_sev_inst = self.time_readout_sev(attn_output) # [B*N, D]
            w_time_sev = None

        # ── (5) [B,N,D]로 재구성 ───────────────────────────────────
        D = z_sev_inst.shape[-1]
        feats_sev  = z_sev_inst.view(B, N, D)    # [B,N,D]

        # ── (6) MIL 집계 (인스턴스 → bag) ──────────────────────────
        sev_logits,  alpha_inst_sev  = self.mil_sev(feats_sev,  mask)  # [B,S], [B,N]
        L = w_time_sev.shape[1] if self.time_pool == "attn" else None
        # 해석용 가중치 (시간/인스턴스)
        aux = {
            "alpha_inst_sev":  alpha_inst_sev,                     # [B,N]
            "w_time_sev":  None if w_time_sev is None else w_time_sev.view(B, N, L),               # [B,N,T]
        }
        return sev_logits, aux

## MIL_Single

In [ ]:
class MIL_Single(nn.Module):
    def __init__(
        self,
        base_encoder: nn.Module,
        num_classes: int,
        *,
        d_model: int = 128,
        time_dropout: float = 0.1,
        attn_dim: int = 128,
        mil_dropout: float = 0.1,
    ):
        super().__init__()
        self.encoder = base_encoder
        self.time_readout = AttnTimeReadout(d_model=d_model, p=time_dropout)  # x:[B*,T,D]
        self.mil_head = AttnMILHead(feat_dim=d_model, num_classes=num_classes,
                                    attn_dim=attn_dim, dropout=mil_dropout)

    def encode_instances(self, x_bag: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        x_bag: [B, N, T, C]
        return:
          z_inst: [B, N, D]
          w_time: [B, N, T]
        """
        B, N, T, C = x_bag.shape
        x_flat = x_bag.reshape(B * N, T, C)
        feat_map = self.encoder.encode(x_flat)
        z_flat, w_flat = self.time_readout(feat_map)  # [B*N, D], [B*N, T]

        T_enc = w_flat.size(-1)         # ★ 인코더가 만든 실제 시간 길이
        D = z_flat.size(-1)             # 임베딩 차원
        # bag 차원 복원
        z_inst = z_flat.view(B, N, -1)    # [B, N, D]
        w_time = w_flat.view(B, N, T_enc)     # [B, N, T]
        return z_inst, w_time

    def forward(
        self,
        x_bag: torch.Tensor,                 # [B, N, T, C]
        mask: Optional[torch.Tensor] = None  # [B, N] (True=valid)
    ):
        z_inst, w_time = self.encode_instances(x_bag)          # [B,N,D], [B,N,T]
        logits_bag, alpha_inst = self.mil_head(z_inst, mask)   # [B,C], [B,N]
        return logits_bag, alpha_inst, w_time

# Dataset

## build manifest

In [ ]:
def build_manifest(
    label_csv_path: Path,
    root_dir: Path,                  # 절대경로가 이미 들어있지만 인터페이스는 유지
    *,
    target_col = 'target',
    require_exists: bool = True,     # True면 실제 파일 존재하는 행만 남김
) -> pd.DataFrame:

    df = pd.read_csv(label_csv_path)

    required = {"file", "patient_id", "session", "task", target_col}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"다음 컬럼이 필요합니다: {missing}")

    rows = []
    for _, r in tqdm(df.iterrows(), total=len(df), desc="Build manifest (K=3)", unit="row"):
        # 1) 경로
        raw_file = str(r["file"]).strip()
        if not raw_file or raw_file.lower() == "nan":
            continue
        p = Path(raw_file)
        if not p.is_absolute():
            p = (root_dir / p).resolve()
        if require_exists and not p.exists():
            continue

        # 2) target (정수)
        try:
            target = int(r[target_col])
        except Exception:
            target = int(float(r[target_col]))

        # 3) patient_id: 'ET01' -> 1
        pid_match = re.search(r"(\d+)", str(r["patient_id"]))
        patient_id = int(pid_match.group(1)) if pid_match else -1

        # 4) session, date: 'ET01_1_01232017' → session=1, date='01232017'
        sess_str = str(r["session"])
        m = _DIR_RE.search(sess_str) or _DIR_RE.search(str(p))
        if m:
            session = int(m.group(2))
            date = m.group(3)
        else:
            session = -1
            date = "00000000"

        # 5) task & task_idx
        task = str(r["task"])
        task_idx = TASK2IDX.get(task.lower(), TASK2IDX["unknown"])

        rows.append(dict(
            path=p,
            patient_id=patient_id,
            session=session,
            date=date,
            task=task,
            task_idx=task_idx,
            target=target,
        ))

    man = pd.DataFrame(rows)
    if len(man):
        # [수정됨] target 컬럼이 0-based가 되도록 조정
        min_target = man["target"].min()
        if min_target > 0:
            man["target"] = man["target"] - min_target

        man = man.sort_values(["patient_id","session","task","path"]).reset_index(drop=True)
        man["target"] = man["target"].astype(int)
    return man


def split_train_valid_by_patient(
    manifest_df: pd.DataFrame,
    val_pid: Sequence[int],
    *,
    pid_col: str = "patient_id",
    reset_index: bool = True
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    manifest 형태의 DataFrame을 받아, val_pid에 포함된 환자는 valid_df로,
    나머지는 train_df로 분리하여 반환합니다.

    Parameters
    ----------
    manifest_df : pd.DataFrame
        'patient_id' 컬럼(기본값)을 포함한 manifest 데이터프레임.
    val_pid : Sequence[int]
        검증 셋에 넣을 환자 번호 리스트. 예) [2, 5, 9]
    pid_col : str, default 'patient_id'
        환자 ID가 들어있는 컬럼명.
    reset_index : bool, default True
        반환하는 데이터프레임의 인덱스를 0부터 재설정할지 여부.

    Returns
    -------
    train_df : pd.DataFrame
        val_pid에 속하지 않는 환자들의 샘플.
    valid_df : pd.DataFrame
        val_pid에 속하는 환자들의 샘플.
    """
    if pid_col not in manifest_df.columns:
        raise KeyError(f"'{pid_col}' 컬럼이 없습니다. 현재 컬럼: {list(manifest_df.columns)}")

    # 숫자형으로 안전 변환 (NaN은 검증 집합에 포함되지 않음)
    pid_series = pd.to_numeric(manifest_df[pid_col], errors="coerce")
    val_set = set(int(x) for x in val_pid)

    valid_mask = pid_series.isin(val_set)
    valid_df = manifest_df.loc[valid_mask].copy()
    train_df = manifest_df.loc[~valid_mask].copy()

    if reset_index:
        valid_df.reset_index(drop=True, inplace=True)
        train_df.reset_index(drop=True, inplace=True)

    return train_df, valid_df

def build_segment_manifest(
    manifest_df: pd.DataFrame,
    *,
    seg_len: int,
    hop: int,
    usecols: Sequence[str] = ("accel_x","accel_y","accel_z","coor_x","coor_y"),
    require_exists: bool = True,
    keep_tail: bool = False,   # 마지막 남는 길이를 버리지 않고 패딩해 포함하고 싶으면 True (패딩은 하지 않고 end=T로만 기록)
) -> pd.DataFrame:
    """
    Parameters
    ----------
    manifest_df : pd.DataFrame
        build_manifest() 결과. 필수 컬럼:
        ['path','patient_id','session','date','task','task_idx','target']
    seg_len : int
        세그먼트 길이(샘플 수)
    hop : int
        홉(겹침). 0 또는 음수면 seg_len으로 간주(오버랩 없음)
    usecols : Sequence[str]
        CSV에서 길이 확인을 위해 읽을 컬럼(존재하는 것만 사용)
    require_exists : bool
        True면 파일이 실제로 있는 행만 처리
    keep_tail : bool
        True면 (T - seg_len) % hop 잔여가 있을 때 마지막 구간을 하나 더 만듦
        (end는 T로 기록; 모델 입력에서 패딩은 사용자가 처리)

    Returns
    -------
    seg_manifest : pd.DataFrame
        컬럼: ['path','patient_id','session','date','task','task_idx','target','start','end','seg_idx']
    """
    required = {"path","patient_id","session","date","task","task_idx","target"}
    missing = required - set(manifest_df.columns)
    if missing:
        raise KeyError(f"다음 컬럼이 필요합니다: {missing}")

    rows = []
    H = seg_len if (hop is None or hop <= 0) else int(hop)

    for _, r in tqdm(manifest_df.iterrows(), total=len(manifest_df), desc="Expand to segments", unit="file"):
        p = Path(str(r["path"]))
        if require_exists and not p.exists():
            continue

        # 존재하는 usecols만 선택해서 가볍게 읽기
        try:
            hdr = pd.read_csv(p, nrows=0)
            cols = [c for c in usecols if c in hdr.columns]
            if len(cols) == 0:
                # 컬럼명이 다르더라도 길이만 알면 되므로 전체 길이만 읽기
                T = len(pd.read_csv(p))
            else:
                tmp = pd.read_csv(p, usecols=cols)
                T = len(tmp)
        except Exception:
            continue

        if T <= 0:
            continue

        # 시작 인덱스들
        starts = list(range(0, max(0, T - seg_len + 1), H))
        if keep_tail and (T > seg_len):
            last_start = starts[-1] if len(starts) else 0
            last_end = last_start + seg_len
            if last_end < T:  # 잔여가 있으면 하나 더
                starts.append(T - seg_len)

        # 너무 짧으면 스킵
        if T < seg_len and not keep_tail:
            continue
        if T < seg_len and keep_tail:
            starts = [0]  # 하나만 만들고 end=T로 설정

        for j, s in enumerate(starts):
            start = int(s)
            end = int(min(s + seg_len, T))  # keep_tail=True일 때 T로 잘릴 수 있음
            rows.append(dict(
                path=str(p),
                patient_id=int(r["patient_id"]),
                session=int(r["session"]),
                date=str(r["date"]),
                task=str(r["task"]),
                task_idx=int(r["task_idx"]),
                target=int(r["target"]),
                start=start,
                end=end,
                seg_idx=j,
            ))

    seg = pd.DataFrame(rows)
    if len(seg):
        seg = seg.sort_values(["patient_id","session","task","path","start"]).reset_index(drop=True)
        # 타입 안전화
        for c in ["patient_id","session","task_idx","target","start","end","seg_idx"]:
            if c in seg.columns:
                seg[c] = seg[c].astype(int, errors="ignore")
    return seg


# 편의 래퍼: train/valid trial manifest를 바로 segment manifest로 확장
def expand_train_valid_to_segments(
    train_manifest: pd.DataFrame,
    valid_manifest: pd.DataFrame,
    *,
    seg_len: int,
    hop: int,
    usecols: Sequence[str] = ("accel_x","accel_y","accel_z","coor_x","coor_y"),
    require_exists: bool = True,
    keep_tail: bool = False,
):
    seg_train = build_segment_manifest(
        train_manifest, seg_len=seg_len, hop=hop, usecols=usecols,
        require_exists=require_exists, keep_tail=keep_tail,
    )
    seg_valid = build_segment_manifest(
        valid_manifest, seg_len=seg_len, hop=hop, usecols=usecols,
        require_exists=require_exists, keep_tail=keep_tail,
    )
    return seg_train, seg_valid

## augmentation

In [ ]:
def _rand_unit_vec3(rng):
    v = rng.standard_normal(3)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rot_R3_axis_angle(axis, theta):
    x,y,z = axis
    c, s = math.cos(theta), math.sin(theta)
    C = 1 - c
    # Rodrigues' rotation formula
    return np.array([
        [c+x*x*C,   x*y*C - z*s, x*z*C + y*s],
        [y*x*C + z*s, c+y*y*C,   y*z*C - x*s],
        [z*x*C - y*s, z*y*C + x*s, c+z*z*C]
    ], dtype=np.float32)

def _rot_R2(theta):
    c, s = math.cos(theta), math.sin(theta)
    return np.array([[c,-s],[s,c]], dtype=np.float32)

class BagRotator:
    """
    bag 전체에 동일 회전(라벨 보존 관점에서 센서/좌표계 변화만 반영)
    - acc_bag: [N,3,T], traj_bag: [N,2,T] (torch.float32)
    - prob: 회전 적용 확률 (기본 0.5)
    """
    def __init__(self, seed=123, prob: float = 0.5):
        self.rng = np.random.default_rng(seed)
        self.prob = float(prob)

    def __call__(self, acc_bag: torch.Tensor, traj_bag: torch.Tensor):
        # 코인플립: 회전 적용 여부
        if self.rng.random() < self.prob:
            # ACC 3D 임의 회전
            axis = _rand_unit_vec3(self.rng)
            theta3 = self.rng.uniform(-math.pi, math.pi)
            R3 = torch.from_numpy(_rot_R3_axis_angle(axis, theta3)).to(acc_bag)

            # TRAJ 2D 임의 회전
            theta2 = self.rng.uniform(-math.pi, math.pi)
            R2 = torch.from_numpy(_rot_R2(theta2)).to(traj_bag)

            if acc_bag.numel():
                acc_bag = torch.einsum("ij,njt->nit", R3, acc_bag)  # [N,3,T]
            if traj_bag.numel():
                traj_bag = torch.einsum("ij,njt->nit", R2, traj_bag) # [N,2,T]
        # else: 그대로 반환
        return acc_bag, traj_bag

def _permute_chunks(x: np.ndarray, N: int, rng: np.random.Generator) -> np.ndarray:
    """
    x: (C,T). N개 동일 길이 청크로 쪼개 무작위 순열. 마지막 청크는 잔여 포함.
    """
    C, T = x.shape
    if N <= 1: return x
    L = T // N
    idxs = [slice(i*L, (i+1)*L) for i in range(N-1)] + [slice((N-1)*L, T)]
    order = np.arange(N); rng.shuffle(order)
    return np.concatenate([x[:, idxs[i]] for i in order], axis=1)

def _timewarp_indices(T: int, rng: np.random.Generator,
                      a_range=(0.15, 0.35), f_range=(0.5, 1.5)):
    """
    속도곡선 v(t)=1 + a*sin(2π f t/T + φ), 누적으로 단조증가 τ(t) 생성 후 0..T-1로 정규화
    반환: τ (length T, strictly increasing)
    """
    t = np.arange(T, dtype=np.float32)
    a = rng.uniform(*a_range)          # amplitude of speed
    f = rng.uniform(*f_range)          # frequency (cycles per window)
    phi = rng.uniform(0, 2*math.pi)    # phase
    v = 1.0 + a * np.sin(2*math.pi*f*t/T + phi)
    v = np.maximum(v, 1e-3)            # positive speed
    tau = np.cumsum(v)
    tau = (tau - tau[0]) / (tau[-1] - tau[0]) * (T - 1)
    return tau.astype(np.float32)

def _interp1(x: np.ndarray, tau: np.ndarray) -> np.ndarray:
    """ x: (C,T), tau: (T,) old->new 매핑에서 x_new[t]=x_old[tau[t]] """
    C, T = x.shape
    t = np.arange(T, dtype=np.float32)
    y = np.empty_like(x)
    for c in range(C):
        y[c] = np.interp(t, tau, x[c])
    return y

class InstancePermTimeW:
    """
    인스턴스 단위 Perm + TimeW (가속도/궤적에 동일 연산으로 동기 유지)
    - Perm: N in [n_min..n_max], 확률 prob_perm로 적용
    - TimeW: 확률 prob_timew로 적용
    """
    def __init__(self, seed=42, n_min=1, n_max=5,
                 tw_a=(0.15,0.35), tw_f=(0.5,1.5),
                 apply_perm=True, apply_timew=True,
                 prob_perm: float = 0.5, prob_timew: float = 0.5):
        self.rng = np.random.default_rng(seed)
        self.n_min = n_min
        self.n_max = n_max
        self.tw_a = tw_a
        self.tw_f = tw_f
        self.apply_perm = apply_perm
        self.apply_timew = apply_timew
        self.prob_perm = float(prob_perm)
        self.prob_timew = float(prob_timew)

    def __call__(self, acc_3xt: np.ndarray, traj_2xt: np.ndarray):
        a = acc_3xt.copy(); t = traj_2xt.copy()
        C1, T = a.shape

        # 1) Permutation (동일 순서 적용)
        do_perm = self.apply_perm and (self.rng.random() < self.prob_perm)
        if do_perm:
            N = int(self.rng.integers(self.n_min, self.n_max + 1))
            if N > 1:
                L = T // N
                idxs = [slice(i*L, (i+1)*L) for i in range(N-1)] + [slice((N-1)*L, T)]
                order = np.arange(N)
                self.rng.shuffle(order)  # 동일 order를 a,t에 같이 사용
                def _perm(x):
                    return np.concatenate([x[:, idxs[i]] for i in order], axis=1)
                a = _perm(a)
                t = _perm(t)

        # 2) TimeWarp (동일 tau 적용)
        do_timew = self.apply_timew and (self.rng.random() < self.prob_timew)
        if do_timew:
            tau = _timewarp_indices(T, self.rng, a_range=self.tw_a, f_range=self.tw_f)
            a = _interp1(a, tau)
            t = _interp1(t, tau)

        return a, t

class _BaseMILLoaderPath:
    """
    seg_df: [path,start,end,target,task,task_idx] (+메타)
    bag 단위: path (CSV 하나가 한 bag)
    """
    def __init__(self, seg_df: pd.DataFrame, target_col: str = "target",
                 usecols: List[str]=USECOLS, traj_div: float = 400.0,
                 filter_task: Optional[str] = None): # ▼▼▼ NEW/MODIFIED ▼▼▼: task 필터링 옵션 추가
        need = {"path","start","end",target_col,"task","task_idx"}
        miss = need - set(seg_df.columns)
        if miss: raise KeyError(f"필수 컬럼 누락: {miss}")

        self.df = seg_df.reset_index(drop=True).copy()
        self.target_col = target_col
        self.usecols = usecols
        self.traj_div = float(traj_div)
        self.filter_task = filter_task # ▼▼▼ NEW/MODIFIED ▼▼▼: 옵션 저장

        # ▼▼▼ NEW/MODIFIED ▼▼▼: 지정된 task가 있으면 데이터프레임 필터링
        if self.filter_task:
            self.df = self.df[self.df["task"] == self.filter_task].copy()
            print(f"[INFO] Dataset filtered for task: '{self.filter_task}'. Total segments: {len(self.df)}")
        # ▲▲▲ NEW/MODIFIED ▲▲▲

        self.df["_bagkey"] = self.df["path"].astype(str)
        self.groups = self.df.groupby("_bagkey")
        self.bags = []
        for k, g in self.groups:
            bag_y = int(g[self.target_col].mode().iloc[0])
            bag_task_idx = int(g["task_idx"].mode().iloc[0])
            bag_task_str = str(g["task"].mode().iloc[0])
            self.bags.append({
                "bagkey": k,
                "rows": g.sort_values("start"),
                "target": bag_y,
                "task_idx": bag_task_idx,
                "task": bag_task_str,
            })

    def load_bag(self, bag_idx: int):
        rows = self.bags[bag_idx]["rows"]
        acc_list, traj_list = [], []
        for _, r in rows.iterrows():
            d = pd.read_csv(r["path"], usecols=self.usecols)
            seg = d.iloc[int(r["start"]):int(r["end"])]
            acc  = seg[["accel_x","accel_y","accel_z"]].to_numpy(float).T
            traj = seg[["coor_x","coor_y"]].to_numpy(float).T / self.traj_div
            acc_list.append(torch.from_numpy(acc).float())
            traj_list.append(torch.from_numpy(traj).float())
        acc_bag  = torch.stack(acc_list, dim=0)
        traj_bag = torch.stack(traj_list, dim=0)
        return (
            acc_bag,
            traj_bag,
            int(self.bags[bag_idx]["target"]),
            int(self.bags[bag_idx]["task_idx"]),
            str(self.bags[bag_idx]["task"]),
            str(self.bags[bag_idx]["bagkey"]),
        )


# ---------- (D) 학습용: 원본 보존 + 증강 복제본으로 클래스 균형화 (수정됨) ----------
class BalancedMILDatasetPath_RPT(Dataset):
    """
    - 원본 bag(aug=False)은 그대로 유지 (allow_downsample=False일 때)
    - 클래스 비율 균등 유지
    - target_per_class 또는 target_total_bags로 최종 bag 수를 제어(원본 보존 제약 하에서)
    """
    def __init__(self, seg_train_df: pd.DataFrame,
                 target_col: str = "target",
                 bag_rot: Optional[BagRotator] = None,
                 inst_perm_timew: Optional[InstancePermTimeW] = None,
                 max_instances_per_bag: Optional[int] = None,
                 print_stats: bool = True,
                 target_per_class: Optional[int] = None,
                 target_total_bags: Optional[int] = None,
                 allow_downsample: bool = False,
                 seed_balance: int = 2024,
                 filter_task: Optional[str] = None): # ▼▼▼ NEW/MODIFIED ▼▼▼: task 필터링 옵션 추가
        # ▼▼▼ NEW/MODIFIED ▼▼▼: _BaseMILLoaderPath에 filter_task 전달
        self.base = _BaseMILLoaderPath(seg_train_df, target_col=target_col, filter_task=filter_task)
        self.bag_rot = bag_rot
        self.inst_ptw = inst_perm_timew
        self.maxN = max_instances_per_bag

        self.orig_indices = list(range(len(self.base.bags)))
        targets = [b["target"] for b in self.base.bags]
        counter = collections.Counter(targets)
        self.classes = sorted(counter.keys())
        self.orig_counts = {c: counter[c] for c in self.classes}
        K = len(self.classes)
        self.max_count = max(counter.values()) if counter else 0

        # --- 목표 개수 계산 (클래스 균등) ---
        if target_per_class is not None:
            desired_per_class = {c: int(target_per_class) for c in self.classes}
        elif target_total_bags is not None and K > 0:
            base = target_total_bags // K
            rem  = target_total_bags % K
            desired_per_class = {c: base + (i < rem) for i, c in enumerate(self.classes)}
        else:
            desired_per_class = {c: self.max_count for c in self.classes}

        if not allow_downsample:
            for c in self.classes:
                if desired_per_class[c] < self.orig_counts[c]:
                    desired_per_class[c] = self.orig_counts[c]

        # --- 아이템 구성: 원본(aug=False) + 증강복제(aug=True) ---
        per_class_indices = {c: [i for i, b in enumerate(self.base.bags) if b["target"] == c] for c in self.classes}
        aug_items = []
        rng = np.random.default_rng(seed_balance)

        kept_original_indices = set(self.orig_indices)
        if allow_downsample:
            new_kept = set()
            for c in self.classes:
                cls_indices = per_class_indices[c]
                keep_n = min(len(cls_indices), desired_per_class[c])
                if keep_n < len(cls_indices):
                    chosen = rng.choice(cls_indices, size=keep_n, replace=False).tolist()
                else:
                    chosen = cls_indices
                new_kept.update(chosen)
            kept_original_indices = new_kept

        for c in self.classes:
            cls_indices = per_class_indices[c]
            kept_n = len([i for i in cls_indices if i in kept_original_indices])
            need = desired_per_class[c] - kept_n
            if need > 0 and kept_n > 0:
                pool = [i for i in cls_indices if i in kept_original_indices]
                sampled = rng.integers(0, len(pool), size=need)
                for s in sampled:
                    aug_items.append((pool[int(s)], True))

        self.items = [(i, False) for i in sorted(kept_original_indices)] + aug_items

        if print_stats and len(self.items) > 0:
            before = dict(self.orig_counts)
            after_counts = collections.Counter([self.base.bags[i]["target"] for (i, is_aug) in self.items if not is_aug])
            total_counts = {c: after_counts.get(c, 0) for c in self.classes}
            for (bi, _) in aug_items:
                c = self.base.bags[bi]["target"]
                total_counts[c] = total_counts.get(c, 0) + 1

            mism = {c: (total_counts[c], desired_per_class[c]) for c in self.classes if total_counts[c] != desired_per_class[c]}
            if mism:
                print("[WARN] Desired per-class counts not exactly met (likely due to original-count constraints):", mism)

            print("[MIL Balance (R+P+T) @ path-bag]")
            print(f"  Task Filter: {self.base.filter_task or 'None'}")
            print("  before :", before)
            print("  desired:", desired_per_class)
            print("  after  :", total_counts, "| total:", len(self.items))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        bag_idx, is_aug = self.items[idx]
        acc_bag, traj_bag, y, y_task, task_str, bagkey = self.base.load_bag(bag_idx)

        if is_aug:
            if self.bag_rot is not None:
                acc_bag, traj_bag = self.bag_rot(acc_bag, traj_bag)
            if self.inst_ptw is not None:
                accs, trajs = [], []
                for j in range(acc_bag.shape[0]):
                    a = acc_bag[j].cpu().numpy()
                    t = traj_bag[j].cpu().numpy()
                    a2, t2 = self.inst_ptw(a, t)
                    accs.append(torch.from_numpy(a2).float())
                    trajs.append(torch.from_numpy(t2).float())
                acc_bag = torch.stack(accs, dim=0)
                traj_bag = torch.stack(trajs, dim=0)

        if (self.maxN is not None) and (acc_bag.shape[0] > self.maxN):
            N = acc_bag.shape[0]
            idx_keep = torch.randperm(N)[:self.maxN]
            acc_bag = acc_bag[idx_keep]
            traj_bag = traj_bag[idx_keep]

        mask = torch.ones(acc_bag.shape[0], dtype=torch.bool)
        meta = {"bagkey": bagkey, "N": acc_bag.shape[0], "aug": bool(is_aug), "task": task_str}
        return {
            "acc_bag": acc_bag,
            "traj_bag": traj_bag,
            "mask": mask,
            "target": y,
            "task_target": y_task,
            "meta": meta
        }


# ---------- (E) 평가용: 원본만 (수정됨) ----------
class EvalMILDatasetPath(Dataset):
    def __init__(self, seg_valid_df: pd.DataFrame, target_col: str="target",
                 filter_task: Optional[str] = None): # ▼▼▼ NEW/MODIFIED ▼▼▼: task 필터링 옵션 추가
        # ▼▼▼ NEW/MODIFIED ▼▼▼: _BaseMILLoaderPath에 filter_task 전달
        self.base = _BaseMILLoaderPath(seg_valid_df, target_col=target_col, filter_task=filter_task)

    def __len__(self):
        return len(self.base.bags)

    def __getitem__(self, i):
        acc_bag, traj_bag, y, y_task, task_str, bagkey = self.base.load_bag(i)
        mask = torch.ones(acc_bag.shape[0], dtype=torch.bool)
        return {
            "acc_bag": acc_bag,
            "traj_bag": traj_bag,
            "mask": mask,
            "target": y,
            "task_target": y_task,
            "meta": {"bagkey": bagkey, "N": acc_bag.shape[0], "aug": False, "task": task_str}
        }


# ---------- (F) collate (변경 없음) ----------
def collate_mil_pad(batch: List[Dict], *, concat_channels: bool = True):
    B = len(batch)
    Ns = [b["acc_bag"].shape[0] for b in batch]
    Cacc = batch[0]["acc_bag"].shape[1]
    Ctraj= batch[0]["traj_bag"].shape[1]
    T    = batch[0]["acc_bag"].shape[2]
    Nmax = max(Ns) if Ns else 0

    def pad_stack(tensors, C):
        out = torch.zeros(B, Nmax, C, T, dtype=tensors[0].dtype)
        for i, x in enumerate(tensors):
            out[i, :x.shape[0]] = x
        return out

    accs  = [b["acc_bag"]  for b in batch]
    trajs = [b["traj_bag"] for b in batch]
    masks = torch.zeros(B, Nmax, dtype=torch.bool)
    ys    = torch.tensor([b["target"] for b in batch], dtype=torch.long)
    ys_task = torch.tensor([b["task_target"] for b in batch], dtype=torch.long)

    for i, b in enumerate(batch):
        masks[i, :Ns[i]] = b["mask"]

    acc_pad  = pad_stack(accs,  Cacc)
    traj_pad = pad_stack(trajs, Ctraj)

    if concat_channels:
        x = torch.cat([acc_pad, traj_pad], dim=2)
        return {
            'x_bag':x,
            'mask':masks,
            'y':ys,
            'y_task':ys_task,
            'meta':[b["meta"] for b in batch]
        }
    else:
        return {
            'acc_bag':acc_pad,
            'traj_bag':traj_pad,
            'mask':masks,
            'y':ys,
            'y_task':ys_task,
            'meta':[b["meta"] for b in batch]
        }

### Rotation

In [ ]:
def _rand_unit_vec3(rng):
    v = rng.standard_normal(3)
    n = np.linalg.norm(v) + 1e-12
    return v / n

def _rot_R3_axis_angle(axis, theta):
    x,y,z = axis
    c, s = math.cos(theta), math.sin(theta)
    C = 1 - c
    # Rodrigues' rotation formula
    return np.array([
        [c+x*x*C,   x*y*C - z*s, x*z*C + y*s],
        [y*x*C + z*s, c+y*y*C,   y*z*C - x*s],
        [z*x*C - y*s, z*y*C + x*s, c+z*z*C]
    ], dtype=np.float32)

def _rot_R2(theta):
    c, s = math.cos(theta), math.sin(theta)
    return np.array([[c,-s],[s,c]], dtype=np.float32)

class BagRotator:
    """
    bag 전체에 동일 회전(라벨 보존 관점에서 센서/좌표계 변화만 반영)
    - acc_bag: [N,3,T], traj_bag: [N,2,T] (torch.float32)
    - prob: 회전 적용 확률 (기본 0.5)
    """
    def __init__(self, seed=123, prob: float = 0.5):
        self.rng = np.random.default_rng(seed)
        self.prob = float(prob)

    def __call__(self, acc_bag: torch.Tensor, traj_bag: torch.Tensor):
        # 코인플립: 회전 적용 여부
        if self.rng.random() < self.prob:
            # ACC 3D 임의 회전
            axis = _rand_unit_vec3(self.rng)
            theta3 = self.rng.uniform(-math.pi, math.pi)
            R3 = torch.from_numpy(_rot_R3_axis_angle(axis, theta3)).to(acc_bag)

            # TRAJ 2D 임의 회전
            theta2 = self.rng.uniform(-math.pi, math.pi)
            R2 = torch.from_numpy(_rot_R2(theta2)).to(traj_bag)

            if acc_bag.numel():
                acc_bag = torch.einsum("ij,njt->nit", R3, acc_bag)  # [N,3,T]
            if traj_bag.numel():
                traj_bag = torch.einsum("ij,njt->nit", R2, traj_bag) # [N,2,T]
        # else: 그대로 반환
        return acc_bag, traj_bag

### Permutation & Time Wrapping

In [ ]:
def _permute_chunks(x: np.ndarray, N: int, rng: np.random.Generator) -> np.ndarray:
    """
    x: (C,T). N개 동일 길이 청크로 쪼개 무작위 순열. 마지막 청크는 잔여 포함.
    """
    C, T = x.shape
    if N <= 1: return x
    L = T // N
    idxs = [slice(i*L, (i+1)*L) for i in range(N-1)] + [slice((N-1)*L, T)]
    order = np.arange(N); rng.shuffle(order)
    return np.concatenate([x[:, idxs[i]] for i in order], axis=1)

def _timewarp_indices(T: int, rng: np.random.Generator,
                      a_range=(0.15, 0.35), f_range=(0.5, 1.5)):
    """
    속도곡선 v(t)=1 + a*sin(2π f t/T + φ), 누적으로 단조증가 τ(t) 생성 후 0..T-1로 정규화
    반환: τ (length T, strictly increasing)
    """
    t = np.arange(T, dtype=np.float32)
    a = rng.uniform(*a_range)          # amplitude of speed
    f = rng.uniform(*f_range)          # frequency (cycles per window)
    phi = rng.uniform(0, 2*math.pi)    # phase
    v = 1.0 + a * np.sin(2*math.pi*f*t/T + phi)
    v = np.maximum(v, 1e-3)            # positive speed
    tau = np.cumsum(v)
    tau = (tau - tau[0]) / (tau[-1] - tau[0]) * (T - 1)
    return tau.astype(np.float32)

def _interp1(x: np.ndarray, tau: np.ndarray) -> np.ndarray:
    """ x: (C,T), tau: (T,) old->new 매핑에서 x_new[t]=x_old[tau[t]] """
    C, T = x.shape
    t = np.arange(T, dtype=np.float32)
    y = np.empty_like(x)
    for c in range(C):
        y[c] = np.interp(t, tau, x[c])
    return y

class InstancePermTimeW:
    """
    인스턴스 단위 Perm + TimeW (가속도/궤적에 동일 연산으로 동기 유지)
    - Perm: N in [n_min..n_max], 확률 prob_perm로 적용
    - TimeW: 확률 prob_timew로 적용
    """
    def __init__(self, seed=42, n_min=1, n_max=5,
                 tw_a=(0.15,0.35), tw_f=(0.5,1.5),
                 apply_perm=True, apply_timew=True,
                 prob_perm: float = 0.5, prob_timew: float = 0.5):
        self.rng = np.random.default_rng(seed)
        self.n_min = n_min
        self.n_max = n_max
        self.tw_a = tw_a
        self.tw_f = tw_f
        self.apply_perm = apply_perm
        self.apply_timew = apply_timew
        self.prob_perm = float(prob_perm)
        self.prob_timew = float(prob_timew)

    def __call__(self, acc_3xt: np.ndarray, traj_2xt: np.ndarray):
        a = acc_3xt.copy(); t = traj_2xt.copy()
        C1, T = a.shape

        # 1) Permutation (동일 순서 적용)
        do_perm = self.apply_perm and (self.rng.random() < self.prob_perm)
        if do_perm:
            N = int(self.rng.integers(self.n_min, self.n_max + 1))
            if N > 1:
                L = T // N
                idxs = [slice(i*L, (i+1)*L) for i in range(N-1)] + [slice((N-1)*L, T)]
                order = np.arange(N)
                self.rng.shuffle(order)  # 동일 order를 a,t에 같이 사용
                def _perm(x):
                    return np.concatenate([x[:, idxs[i]] for i in order], axis=1)
                a = _perm(a)
                t = _perm(t)

        # 2) TimeWarp (동일 tau 적용)
        do_timew = self.apply_timew and (self.rng.random() < self.prob_timew)
        if do_timew:
            tau = _timewarp_indices(T, self.rng, a_range=self.tw_a, f_range=self.tw_f)
            a = _interp1(a, tau)
            t = _interp1(t, tau)

        return a, t

### MIL Bag Dataset

In [ ]:
class _BaseMILLoaderPath:
    """
    seg_df: [path,start,end,target,task,task_idx] (+메타)
    bag 단위: path (CSV 하나가 한 bag)
    """
    def __init__(self, seg_df: pd.DataFrame, target_col: str = "target",
                 usecols: List[str]=USECOLS, traj_div: float = 400.0,
                 filter_task: Optional[str] = None): # ▼▼▼ NEW/MODIFIED ▼▼▼: task 필터링 옵션 추가
        need = {"path","start","end",target_col,"task","task_idx"}
        miss = need - set(seg_df.columns)
        if miss: raise KeyError(f"필수 컬럼 누락: {miss}")

        self.df = seg_df.reset_index(drop=True).copy()
        self.target_col = target_col
        self.usecols = usecols
        self.traj_div = float(traj_div)
        self.filter_task = filter_task # ▼▼▼ NEW/MODIFIED ▼▼▼: 옵션 저장

        # ▼▼▼ NEW/MODIFIED ▼▼▼: 지정된 task가 있으면 데이터프레임 필터링
        if self.filter_task:
            self.df = self.df[self.df["task"] == self.filter_task].copy()
            print(f"[INFO] Dataset filtered for task: '{self.filter_task}'. Total segments: {len(self.df)}")
        # ▲▲▲ NEW/MODIFIED ▲▲▲

        self.df["_bagkey"] = self.df["path"].astype(str)
        self.groups = self.df.groupby("_bagkey")
        self.bags = []
        for k, g in self.groups:
            bag_y = int(g[self.target_col].mode().iloc[0])
            bag_task_idx = int(g["task_idx"].mode().iloc[0])
            bag_task_str = str(g["task"].mode().iloc[0])
            self.bags.append({
                "bagkey": k,
                "rows": g.sort_values("start"),
                "target": bag_y,
                "task_idx": bag_task_idx,
                "task": bag_task_str,
            })

    def load_bag(self, bag_idx: int):
        rows = self.bags[bag_idx]["rows"]
        acc_list, traj_list = [], []
        for _, r in rows.iterrows():
            d = pd.read_csv(r["path"], usecols=self.usecols)
            seg = d.iloc[int(r["start"]):int(r["end"])]
            acc  = seg[["accel_x","accel_y","accel_z"]].to_numpy(float).T
            traj = seg[["coor_x","coor_y"]].to_numpy(float).T / self.traj_div
            acc_list.append(torch.from_numpy(acc).float())
            traj_list.append(torch.from_numpy(traj).float())
        acc_bag  = torch.stack(acc_list, dim=0)
        traj_bag = torch.stack(traj_list, dim=0)
        return (
            acc_bag,
            traj_bag,
            int(self.bags[bag_idx]["target"]),
            int(self.bags[bag_idx]["task_idx"]),
            str(self.bags[bag_idx]["task"]),
            str(self.bags[bag_idx]["bagkey"]),
        )


# ---------- (D) 학습용: 원본 보존 + 증강 복제본으로 클래스 균형화 (수정됨) ----------
class BalancedMILDatasetPath_RPT(Dataset):
    """
    - 원본 bag(aug=False)은 그대로 유지 (allow_downsample=False일 때)
    - 클래스 비율 균등 유지
    - target_per_class 또는 target_total_bags로 최종 bag 수를 제어(원본 보존 제약 하에서)
    """
    def __init__(self, seg_train_df: pd.DataFrame,
                 target_col: str = "target",
                 bag_rot: Optional[BagRotator] = None,
                 inst_perm_timew: Optional[InstancePermTimeW] = None,
                 max_instances_per_bag: Optional[int] = None,
                 print_stats: bool = True,
                 target_per_class: Optional[int] = None,
                 target_total_bags: Optional[int] = None,
                 allow_downsample: bool = False,
                 seed_balance: int = 2024,
                 filter_task: Optional[str] = None): # ▼▼▼ NEW/MODIFIED ▼▼▼: task 필터링 옵션 추가
        # ▼▼▼ NEW/MODIFIED ▼▼▼: _BaseMILLoaderPath에 filter_task 전달
        self.base = _BaseMILLoaderPath(seg_train_df, target_col=target_col, filter_task=filter_task)
        self.bag_rot = bag_rot
        self.inst_ptw = inst_perm_timew
        self.maxN = max_instances_per_bag

        self.orig_indices = list(range(len(self.base.bags)))
        targets = [b["target"] for b in self.base.bags]
        counter = collections.Counter(targets)
        self.classes = sorted(counter.keys())
        self.orig_counts = {c: counter[c] for c in self.classes}
        K = len(self.classes)
        self.max_count = max(counter.values()) if counter else 0

        # --- 목표 개수 계산 (클래스 균등) ---
        if target_per_class is not None:
            desired_per_class = {c: int(target_per_class) for c in self.classes}
        elif target_total_bags is not None and K > 0:
            base = target_total_bags // K
            rem  = target_total_bags % K
            desired_per_class = {c: base + (i < rem) for i, c in enumerate(self.classes)}
        else:
            desired_per_class = {c: self.max_count for c in self.classes}

        if not allow_downsample:
            for c in self.classes:
                if desired_per_class[c] < self.orig_counts[c]:
                    desired_per_class[c] = self.orig_counts[c]

        # --- 아이템 구성: 원본(aug=False) + 증강복제(aug=True) ---
        per_class_indices = {c: [i for i, b in enumerate(self.base.bags) if b["target"] == c] for c in self.classes}
        aug_items = []
        rng = np.random.default_rng(seed_balance)

        kept_original_indices = set(self.orig_indices)
        if allow_downsample:
            new_kept = set()
            for c in self.classes:
                cls_indices = per_class_indices[c]
                keep_n = min(len(cls_indices), desired_per_class[c])
                if keep_n < len(cls_indices):
                    chosen = rng.choice(cls_indices, size=keep_n, replace=False).tolist()
                else:
                    chosen = cls_indices
                new_kept.update(chosen)
            kept_original_indices = new_kept

        for c in self.classes:
            cls_indices = per_class_indices[c]
            kept_n = len([i for i in cls_indices if i in kept_original_indices])
            need = desired_per_class[c] - kept_n
            if need > 0 and kept_n > 0:
                pool = [i for i in cls_indices if i in kept_original_indices]
                sampled = rng.integers(0, len(pool), size=need)
                for s in sampled:
                    aug_items.append((pool[int(s)], True))

        self.items = [(i, False) for i in sorted(kept_original_indices)] + aug_items

        if print_stats and len(self.items) > 0:
            before = dict(self.orig_counts)
            after_counts = collections.Counter([self.base.bags[i]["target"] for (i, is_aug) in self.items if not is_aug])
            total_counts = {c: after_counts.get(c, 0) for c in self.classes}
            for (bi, _) in aug_items:
                c = self.base.bags[bi]["target"]
                total_counts[c] = total_counts.get(c, 0) + 1

            mism = {c: (total_counts[c], desired_per_class[c]) for c in self.classes if total_counts[c] != desired_per_class[c]}
            if mism:
                print("[WARN] Desired per-class counts not exactly met (likely due to original-count constraints):", mism)

            print("[MIL Balance (R+P+T) @ path-bag]")
            print(f"  Task Filter: {self.base.filter_task or 'None'}")
            print("  before :", before)
            print("  desired:", desired_per_class)
            print("  after  :", total_counts, "| total:", len(self.items))

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        bag_idx, is_aug = self.items[idx]
        acc_bag, traj_bag, y, y_task, task_str, bagkey = self.base.load_bag(bag_idx)

        if is_aug:
            if self.bag_rot is not None:
                acc_bag, traj_bag = self.bag_rot(acc_bag, traj_bag)
            if self.inst_ptw is not None:
                accs, trajs = [], []
                for j in range(acc_bag.shape[0]):
                    a = acc_bag[j].cpu().numpy()
                    t = traj_bag[j].cpu().numpy()
                    a2, t2 = self.inst_ptw(a, t)
                    accs.append(torch.from_numpy(a2).float())
                    trajs.append(torch.from_numpy(t2).float())
                acc_bag = torch.stack(accs, dim=0)
                traj_bag = torch.stack(trajs, dim=0)

        if (self.maxN is not None) and (acc_bag.shape[0] > self.maxN):
            N = acc_bag.shape[0]
            idx_keep = torch.randperm(N)[:self.maxN]
            acc_bag = acc_bag[idx_keep]
            traj_bag = traj_bag[idx_keep]

        mask = torch.ones(acc_bag.shape[0], dtype=torch.bool)
        meta = {"bagkey": bagkey, "N": acc_bag.shape[0], "aug": bool(is_aug), "task": task_str}
        return {
            "acc_bag": acc_bag,
            "traj_bag": traj_bag,
            "mask": mask,
            "target": y,
            "task_target": y_task,
            "meta": meta
        }


# ---------- (E) 평가용: 원본만 (수정됨) ----------
class EvalMILDatasetPath(Dataset):
    def __init__(self, seg_valid_df: pd.DataFrame, target_col: str="target",
                 filter_task: Optional[str] = None): # ▼▼▼ NEW/MODIFIED ▼▼▼: task 필터링 옵션 추가
        # ▼▼▼ NEW/MODIFIED ▼▼▼: _BaseMILLoaderPath에 filter_task 전달
        self.base = _BaseMILLoaderPath(seg_valid_df, target_col=target_col, filter_task=filter_task)

    def __len__(self):
        return len(self.base.bags)

    def __getitem__(self, i):
        acc_bag, traj_bag, y, y_task, task_str, bagkey = self.base.load_bag(i)
        mask = torch.ones(acc_bag.shape[0], dtype=torch.bool)
        return {
            "acc_bag": acc_bag,
            "traj_bag": traj_bag,
            "mask": mask,
            "target": y,
            "task_target": y_task,
            "meta": {"bagkey": bagkey, "N": acc_bag.shape[0], "aug": False, "task": task_str}
        }


# ---------- (F) collate (변경 없음) ----------
def collate_mil_pad(batch: List[Dict], *, concat_channels: bool = True):
    B = len(batch)
    Ns = [b["acc_bag"].shape[0] for b in batch]
    Cacc = batch[0]["acc_bag"].shape[1]
    Ctraj= batch[0]["traj_bag"].shape[1]
    T    = batch[0]["acc_bag"].shape[2]
    Nmax = max(Ns) if Ns else 0

    def pad_stack(tensors, C):
        out = torch.zeros(B, Nmax, C, T, dtype=tensors[0].dtype)
        for i, x in enumerate(tensors):
            out[i, :x.shape[0]] = x
        return out

    accs  = [b["acc_bag"]  for b in batch]
    trajs = [b["traj_bag"] for b in batch]
    masks = torch.zeros(B, Nmax, dtype=torch.bool)
    ys    = torch.tensor([b["target"] for b in batch], dtype=torch.long)
    ys_task = torch.tensor([b["task_target"] for b in batch], dtype=torch.long)

    for i, b in enumerate(batch):
        masks[i, :Ns[i]] = b["mask"]

    acc_pad  = pad_stack(accs,  Cacc)
    traj_pad = pad_stack(trajs, Ctraj)

    if concat_channels:
        x = torch.cat([acc_pad, traj_pad], dim=2)
        return {
            'x_bag':x,
            'mask':masks,
            'y':ys,
            'y_task':ys_task,
            'meta':[b["meta"] for b in batch]
        }
    else:
        return {
            'acc_bag':acc_pad,
            'traj_bag':traj_pad,
            'mask':masks,
            'y':ys,
            'y_task':ys_task,
            'meta':[b["meta"] for b in batch]
        }

# Functions for E

## model builder

In [ ]:
def build_model(
    model_name: str,
    *,
    in_ch: int,
    num_classes: int,
    # 공통/기본 하이퍼파라미터 (필요 모델에만 사용)
    seq_len: int = SEG_LEN,
    d_model: int = 128,
    hidden_size: int = 128,
    num_layers: int = 2,
    dropout: float = 0.1,
    # MyWaveNet
    dilations: list = (1, 2, 4, 8, 16, 32),
    n_stacks: int = 2,
    # TimesNet
    d_ff: int = 128,
    e_layers: int = 1,
    top_k: int = 5,
    f_min: float = None,
):
    """
    model_name ∈ {'LSTM', 'ResNet18', 'TimesNet', 'MyWaveNet'}
    입력 텐서 형식은 통일하여 [B, T, C]를 기대합니다.
    """
    name = model_name.strip().lower()

    if name == 'lstm':
        model = LSTM(in_ch, hidden_size, num_layers, dropout=dropout)
        return model

    elif name == 'resnet18':
        model = ResNet18_1D(num_classes=num_classes,
                            input_channels=in_ch,
                            d_model=d_model)
        return model

    elif name == 'timesnet':
        assert seq_len is not None, "TimesNet을 쓰려면 seq_len을 지정하세요."
        model = TimesNet(
            num_class=num_classes,
            seq_len=seq_len,
            enc_in=in_ch,
            d_model=d_model,
            d_ff=d_ff,
            e_layers=e_layers,
            top_k=top_k,
            f_min=f_min,
            dropout=dropout
        )
        return model

    elif name == 'mywavenet':
        model = MyWaveNet(
            in_ch=in_ch,
            res_ch=d_model,
            skip_ch=d_model,
            dilations=list(dilations),
            n_classes=num_classes,
            n_stacks=n_stacks
        )
        return model

    else:
        raise ValueError(f"알 수 없는 model_name: {model_name!r}. "
                         f"['LSTM','ResNet18','TimesNet','MyWaveNet'] 중에서 선택하세요.")

## Multimodal Train Function

In [ ]:
def _compute_metrics(logits: torch.Tensor, labels: torch.Tensor) -> Dict[str, float]:
    """
    Args:
        logits: [B, C]
        labels: [B]
    Returns:
        dict with:
         - acc, correct, total
         - conf: [C,C] confusion matrix (rows=GT, cols=Pred)
    """
    with torch.no_grad():
        preds = logits.argmax(dim=1)
        correct = (preds == labels).sum().item()
        total = labels.numel()
        acc = correct / max(1, total)

        C = logits.size(1)
        # rows = GT, cols = Pred
        idx = (labels.long() * C + preds.long()).view(-1)
        conf = torch.bincount(idx, minlength=C * C).reshape(C, C)

    return {"acc": acc, "correct": float(correct), "total": float(total), "conf": conf}


def _confmat_to_perclass_acc_macro_f1(conf: torch.Tensor) -> Dict[str, object]:
    """
    # ▼▼▼ MODIFIED ▼▼▼
    # Tensors are now created on the same device as the input 'conf' tensor to prevent runtime errors.
    # GT(Ground Truth)에 등장한 클래스에 대해서만 macro_f1을 계산하고,
    # 등장하지 않은 클래스의 per-class acc는 NaN으로 표시합니다.
    # ▲▲▲ MODIFIED ▲▲▲
    """
    conf = conf.float()
    C = conf.size(0)
    if C == 0:
        return {"per_class_acc": [], "macro_f1": 0.0}

    device = conf.device  # Get device from input tensor
    support = conf.sum(dim=1)      # GT count per class
    pred_cnt = conf.sum(dim=0)     # Pred count per class
    tp = conf.diag()

    # per-class accuracy (= recall per class)
    # Create tensor on the correct device
    per_class_acc = torch.full((C,), float('nan'), dtype=torch.float32, device=device)
    mask_sup = support > 0
    per_class_acc[mask_sup] = tp[mask_sup] / support[mask_sup]

    # precision, recall, f1 per class
    # Create tensors on the correct device
    precision = torch.zeros(C, dtype=torch.float32, device=device)
    recall    = torch.zeros(C, dtype=torch.float32, device=device)
    precision[pred_cnt > 0] = tp[pred_cnt > 0] / pred_cnt[pred_cnt > 0]
    recall[mask_sup]        = tp[mask_sup] / support[mask_sup]

    denom = precision + recall
    f1 = torch.zeros(C, dtype=torch.float32, device=device)
    nz = denom > 0
    f1[nz] = 2 * precision[nz] * recall[nz] / denom[nz]

    # GT에 등장한 클래스(support > 0)에 대해서만 F1 평균 계산
    macro_f1 = float(f1[mask_sup].mean().item()) if mask_sup.any() else 0.0

    return {
        "per_class_acc": per_class_acc.cpu().tolist(),
        "macro_f1": macro_f1,
    }


@torch.no_grad()
def evaluate_on_loader(
    model: nn.Module,
    loader,
    device: torch.device,
    *,
    target: str = "severity",
    modality: Optional[str] = None
) -> Dict[str, object]:
    """
    선택된 타깃/모달리티로 평가합니다.
    (내부 로직은 변경 없음, _confmat_to_perclass_acc_macro_f1와 로직 일관성 확인)
    """
    assert target in ("severity", "task"), "target must be 'severity' or 'task'"
    assert modality in (None, "acc", "traj"), "modality must be None, 'acc', or 'traj'"

    # 내부 메트릭 계산 함수. _confmat_to_perclass_acc_macro_f1와 동일한 로직 사용.
    def _metrics_from_conf(conf: torch.Tensor):
        conf = conf.to(torch.float32)
        C = conf.size(0)
        support = conf.sum(dim=1)
        pred_cnt = conf.sum(dim=0)
        tp = torch.diag(conf)

        per_class_acc = torch.full((C,), float('nan'), dtype=torch.float32)
        mask_sup = support > 0
        per_class_acc[mask_sup] = tp[mask_sup] / support[mask_sup]

        precision = torch.zeros(C, dtype=torch.float32)
        recall    = torch.zeros(C, dtype=torch.float32)
        precision[pred_cnt > 0] = tp[pred_cnt > 0] / pred_cnt[pred_cnt > 0]
        recall[mask_sup]        = tp[mask_sup] / support[mask_sup]

        denom = precision + recall
        f1 = torch.zeros(C, dtype=torch.float32)
        nz = denom > 0
        f1[nz] = 2 * precision[nz] * recall[nz] / denom[nz]

        macro_f1 = float(f1[mask_sup].mean().item()) if mask_sup.any() else 0.0
        return {"macro_f1": macro_f1, "per_class_acc": per_class_acc.tolist()}

    model.eval()
    epoch_conf = None
    correct = 0.0
    total = 0.0

    pbar = tqdm(loader, desc=f"Eval({target}|{modality or 'both'})", leave=False)
    for batch in pbar:
        acc  = batch["acc_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)
        traj = batch["traj_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)
        mask = batch["mask"].to(device, non_blocking=True)
        y = (batch["y"] if target == "severity" else batch["y_task"]).to(device, non_blocking=True)

        if modality is None:
            out = model(acc, traj, mask=mask)
            if isinstance(out, (tuple, list)) and len(out) >= 2:
                sev_logits, task_logits = out[0], out[1]
                logits = sev_logits if target == "severity" else task_logits
            else:
                logits = out if torch.is_tensor(out) else out[0]
        elif modality == "acc":
            out = model(acc, mask=mask)
            logits = out[0] if (isinstance(out, (tuple, list)) and len(out) > 0) else out
        else:
            out = model(traj, mask=mask)
            logits = out[0] if (isinstance(out, (tuple, list)) and len(out) > 0) else out

        preds = logits.argmax(dim=-1)
        y_cpu = y.detach().cpu()
        p_cpu = preds.detach().cpu()

        C = logits.size(-1)
        if epoch_conf is None:
            epoch_conf = torch.zeros(C, C, dtype=torch.long)

        for c in range(C):
            m = (y_cpu == c)
            if m.any():
                binc = torch.bincount(p_cpu[m], minlength=C)
                epoch_conf[c, :] += binc

        correct += (p_cpu == y_cpu).sum().item()
        total   += float(y_cpu.numel())

        mm_now = _metrics_from_conf(epoch_conf)
        acc_now = correct / max(1.0, total)
        pbar.set_postfix({"acc": f"{acc_now:.3f}", "macro_f1": f"{mm_now['macro_f1']:.3f}"})

    if epoch_conf is None:
        return {"acc": 0.0, "macro_f1": 0.0, "per_class_acc": [], "conf": torch.zeros(0, 0)}

    acc_final = correct / max(1.0, total)
    mm = _metrics_from_conf(epoch_conf)
    return {
        "acc": float(acc_final),
        "macro_f1": float(mm["macro_f1"]),
        "per_class_acc": mm["per_class_acc"],
        "conf": epoch_conf,
    }


def _run_epoch(
    model: nn.Module,
    loader,
    device: torch.device,
    criterion: nn.Module,
    optimizer: Optional[torch.optim.Optimizer] = None,
    grad_clip: Optional[float] = 1.0,
    lamda: float = 0.3,
    task_class_weights: Optional[torch.Tensor] = None # 'tw' -> 'task_class_weights'로 명확화
) -> Dict[str, float]:
    is_train = optimizer is not None
    model.train(mode=is_train)
    desc = "Train" if is_train else "Valid"

    tot_loss, tot_sev_loss, tot_task_loss = 0.0, 0.0, 0.0
    n_batches = 0
    sev_correct, sev_total = 0.0, 0.0
    task_correct, task_total = 0.0, 0.0
    epoch_conf_sev, epoch_conf_task = None, None

    pbar = tqdm(loader, desc=desc, leave=False)
    for batch in pbar:
        acc  = batch["acc_bag"].permute(0, 1, 3, 2).to(device)
        traj = batch["traj_bag"].permute(0, 1, 3, 2).to(device)
        mask = batch["mask"].to(device)
        y    = batch["y"].to(device)
        y_t  = batch["y_task"].to(device)

        with torch.set_grad_enabled(is_train):
            sev_logits, task_logits, _ = model(acc, traj, mask=mask)
            sev_loss = criterion(sev_logits, y)

            task_loss = F.cross_entropy(task_logits, y_t, weight=task_class_weights)

            loss = sev_loss + lamda * task_loss

            if is_train:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                if grad_clip is not None:
                    nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()

        sm = _compute_metrics(sev_logits.detach(), y.detach())
        tm = _compute_metrics(task_logits.detach(), y_t.detach())

        epoch_conf_sev = sm["conf"].clone() if epoch_conf_sev is None else epoch_conf_sev + sm["conf"]
        epoch_conf_task = tm["conf"].clone() if epoch_conf_task is None else epoch_conf_task + tm["conf"]

        sev_correct += sm["correct"]; sev_total += sm["total"]
        task_correct += tm["correct"]; task_total += tm["total"]

        tot_loss += float(loss.detach().item())
        tot_sev_loss += float(sev_loss.detach().item())
        tot_task_loss += float(task_loss.detach().item())
        n_batches += 1

        sev_mm = _confmat_to_perclass_acc_macro_f1(epoch_conf_sev)
        task_mm = _confmat_to_perclass_acc_macro_f1(epoch_conf_task)

        pbar.set_postfix({
            "sev_loss": f"{tot_sev_loss / n_batches:.4f}",
            "task_loss": f"{tot_task_loss / n_batches:.4f}",
            "loss": f"{tot_loss / n_batches:.4f}",
            "sev_acc": f"{(sev_correct / max(1.0, sev_total)):.3f}",
            "task_acc": f"{(task_correct / max(1.0, task_total)):.3f}",
            "sev_f1": f"{sev_mm['macro_f1']:.3f}",
            "task_f1": f"{task_mm['macro_f1']:.3f}",
        })

    sev_mm = _confmat_to_perclass_acc_macro_f1(epoch_conf_sev if epoch_conf_sev is not None else torch.zeros(0,0))
    task_mm = _confmat_to_perclass_acc_macro_f1(epoch_conf_task if epoch_conf_task is not None else torch.zeros(0,0))

    return {
        "loss": tot_loss / max(1, n_batches),
        "sev_loss": tot_sev_loss / max(1, n_batches),
        "task_loss": tot_task_loss / max(1, n_batches),
        "sev_acc": sev_correct / max(1.0, sev_total),
        "task_acc": task_correct / max(1.0, task_total),
        "sev_macro_f1": sev_mm["macro_f1"],
        "task_macro_f1": task_mm["macro_f1"],
        "sev_cls_acc": sev_mm["per_class_acc"],
        "task_cls_acc": task_mm["per_class_acc"],
    }


def train_model(
    model: nn.Module,
    train_loader,
    valid_loader,
    *,
    device: torch.device,
    epochs: int = 30,
    learning_rate: float = 1e-3,
    early_stopping_patience: int = 10,
    weight_decay: float = 1e-4,
    grad_clip: Optional[float] = 1.0,
    lamda: float = 0.3,
    task_class_weights: Optional[torch.Tensor] = None, # 'tw' -> 'task_class_weights'로 명확화
    use_scheduler: bool = True,
    sched_mode: str = "min",
    sched_factor: float = 0.5,
    sched_patience: int = 3,
    sched_cooldown: int = 1,
    sched_min_lr: float = 1e-6,
    sched_threshold: float = 1e-3,
    sched_threshold_mode: str = "rel",
    sched_eps: float = 1e-8,
) -> Dict[str, List[float]]:
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode=sched_mode, factor=sched_factor, patience=sched_patience,
            threshold=sched_threshold, threshold_mode=sched_threshold_mode,
            cooldown=sched_cooldown, min_lr=sched_min_lr, eps=sched_eps,
        )

    def _get_lr(opt): return opt.param_groups[0]["lr"]

    best_val_metric = float("inf") if sched_mode == "min" else float("-inf")
    best_state = None
    patience_counter = 0

    history = {k: [] for k in [
        "train_loss", "train_sev_loss", "train_task_loss", "train_sev_acc", "train_task_acc",
        "valid_loss", "valid_sev_loss", "valid_task_loss", "valid_sev_acc", "valid_task_acc",
        "train_sev_macro_f1", "train_task_macro_f1",
        "valid_sev_macro_f1", "valid_task_macro_f1",
        "train_sev_cls_acc", "train_task_cls_acc",
        "valid_sev_cls_acc", "valid_task_cls_acc", "lr",
    ]}

    for ep in range(1, epochs + 1):
        tr = _run_epoch(model, train_loader, device, criterion, optimizer=optimizer, grad_clip=grad_clip, lamda=lamda, task_class_weights=task_class_weights)
        va = _run_epoch(model, valid_loader, device, criterion, optimizer=None, grad_clip=None, lamda=lamda, task_class_weights=task_class_weights)

        cur_lr = _get_lr(optimizer)
        if scheduler is not None:
            monitor_metric = va["sev_loss"] if sched_mode == "min" else va["sev_macro_f1"]
            scheduler.step(monitor_metric)

        # Record history
        for k, v in tr.items(): history[f"train_{k}"].append(v)
        for k, v in va.items(): history[f"valid_{k}"].append(v)
        history["lr"].append(cur_lr)

        # ▼▼▼ NEW/MODIFIED ▼▼▼: NaN 값을 포함할 수 있는 per-class accuracy 배열을 안전하게 출력
        def fmt_arr(a):
            return "[" + ", ".join(f"{'nan' if np.isnan(x) else f'{x:.3f}'}" for x in a) + "]"
        # ▲▲▲ NEW/MODIFIED ▲▲▲

        print(
            f"[Epoch {ep:03d}] "
            f"TR loss {tr['loss']:.4f} (sev {tr['sev_loss']:.4f}, task {tr['task_loss']:.4f}) | "
            f"VA loss {va['loss']:.4f} (sev {va['sev_loss']:.4f}, task {va['task_loss']:.4f}) | "
            f"lr {history['lr'][-1]:.2e}\n"
            f"       TR acc (sev {tr['sev_acc']:.3f}, task {tr['task_acc']:.3f}) | "
            f"VA acc (sev {va['sev_acc']:.3f}, task {va['task_acc']:.3f})\n"
            f"       TR F1 (sev {tr['sev_macro_f1']:.3f}, task {tr['task_macro_f1']:.3f}) | "
            f"VA F1 (sev {va['sev_macro_f1']:.3f}, task {va['task_macro_f1']:.3f})\n"
            f"       TR per-class acc (sev {fmt_arr(tr['sev_cls_acc'])}, task {fmt_arr(tr['task_cls_acc'])})\n"
            f"       VA per-class acc (sev {fmt_arr(va['sev_cls_acc'])}, task {fmt_arr(va['task_cls_acc'])})"
        )
        print()

        # Early stopping check
        current_metric = va["sev_loss"] if sched_mode == "min" else va["sev_macro_f1"]
        is_better = (current_metric < best_val_metric) if sched_mode == "min" else (current_metric > best_val_metric)

        if is_better:
            best_val_metric = current_metric
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {ep} (no improvement for {early_stopping_patience} epochs).")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return history


In [ ]:
def _unpack_sev_only(model_out):
    if isinstance(model_out, (tuple, list)):
        if len(model_out) == 2:
            sev_logits, aux = model_out
            return sev_logits, aux
        elif len(model_out) == 3:
            sev_logits, _task_logits, aux = model_out
            return sev_logits, aux
    # 모델이 sev_logits만 주는 경우
    return model_out, {}


def _run_epoch_sev_only(
    model: nn.Module,
    loader,
    device: torch.device,
    criterion: nn.Module,
    optimizer: Optional[torch.optim.Optimizer] = None,
    grad_clip: Optional[float] = 1.0,
) -> Dict[str, float]:
    """
    sev(중증도) 단일 과제 학습/검증 루프.
    - task 로짓/로스/메트릭 일체 제거
    - confmat 기반 macro-F1 및 per-class acc 계산 (_compute_metrics, _confmat_to_perclass_acc_macro_f1 사용)
    """
    is_train = optimizer is not None
    model.train(mode=is_train)
    desc = "Train" if is_train else "Valid"

    tot_loss = 0.0
    n_batches = 0
    sev_correct = 0.0
    sev_total = 0.0
    epoch_conf_sev = None

    pbar = tqdm(loader, desc=desc, leave=False)
    for batch in pbar:
        acc  = batch["acc_bag"].permute(0, 1, 3, 2).to(device)
        traj = batch["traj_bag"].permute(0, 1, 3, 2).to(device)
        mask = batch["mask"].to(device)
        y    = batch["y"].to(device)

        with torch.set_grad_enabled(is_train):
            sev_logits, _ = _unpack_sev_only(model(acc, traj, mask=mask))
            sev_loss = criterion(sev_logits, y)

            if is_train:
                optimizer.zero_grad(set_to_none=True)
                sev_loss.backward()
                if grad_clip is not None:
                    nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()

        sm = _compute_metrics(sev_logits.detach(), y.detach())
        epoch_conf_sev = sm["conf"].clone() if epoch_conf_sev is None else epoch_conf_sev + sm["conf"]

        sev_correct += sm["correct"]
        sev_total   += sm["total"]
        tot_loss += float(sev_loss.detach().item())
        n_batches += 1

        sev_mm = _confmat_to_perclass_acc_macro_f1(epoch_conf_sev)
        pbar.set_postfix({
            "sev_loss": f"{tot_loss / n_batches:.4f}",
            "sev_acc":  f"{(sev_correct / max(1.0, sev_total)):.3f}",
            "sev_f1":   f"{sev_mm['macro_f1']:.3f}",
        })

    sev_mm = _confmat_to_perclass_acc_macro_f1(epoch_conf_sev if epoch_conf_sev is not None else torch.zeros(0,0))
    return {
        "loss":         tot_loss / max(1, n_batches),
        "sev_loss":     tot_loss / max(1, n_batches),
        "sev_acc":      sev_correct / max(1.0, sev_total),
        "sev_macro_f1": sev_mm["macro_f1"],
        "sev_cls_acc":  sev_mm["per_class_acc"],
    }


def train_model_sev_only(
    model: nn.Module,
    train_loader,
    valid_loader,
    *,
    device: torch.device,
    epochs: int = 30,
    learning_rate: float = 1e-3,
    early_stopping_patience: int = 10,
    weight_decay: float = 1e-4,
    grad_clip: Optional[float] = 1.0,
    use_scheduler: bool = True,
    sched_mode: str = "min",
    sched_factor: float = 0.5,
    sched_patience: int = 3,
    sched_cooldown: int = 1,
    sched_min_lr: float = 1e-6,
    sched_threshold: float = 1e-3,
    sched_threshold_mode: str = "rel",
    sched_eps: float = 1e-8,
) -> Dict[str, List[float]]:
    """
    Severity 단일 태스크 학습 루프.
    """
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode=sched_mode, factor=sched_factor, patience=sched_patience,
            threshold=sched_threshold, threshold_mode=sched_threshold_mode,
            cooldown=sched_cooldown, min_lr=sched_min_lr, eps=sched_eps,
        )

    def _get_lr(opt): return opt.param_groups[0]["lr"]

    best_val_metric = float("inf") if sched_mode == "min" else float("-inf")
    best_state = None
    patience_counter = 0

    history = {k: [] for k in [
        "train_loss", "train_sev_loss", "train_sev_acc", "train_sev_macro_f1", "train_sev_cls_acc",
        "valid_loss", "valid_sev_loss", "valid_sev_acc", "valid_sev_macro_f1", "valid_sev_cls_acc",
        "lr",
    ]}

    for ep in range(1, epochs + 1):
        tr = _run_epoch_sev_only(model, train_loader, device, criterion, optimizer=optimizer, grad_clip=grad_clip)
        va = _run_epoch_sev_only(model, valid_loader, device, criterion, optimizer=None, grad_clip=None)

        cur_lr = _get_lr(optimizer)
        if scheduler is not None:
            monitor_metric = va["sev_loss"] if sched_mode == "min" else va["sev_macro_f1"]
            scheduler.step(monitor_metric)
        history["lr"].append(cur_lr)

        # 기록
        history_keys = list(tr.keys())
        for k in history_keys:
            if f"train_{k}" in history: history[f"train_{k}"].append(tr[k])
            if f"valid_{k}" in history: history[f"valid_{k}"].append(va[k])

        # ▼▼▼ NEW/MODIFIED ▼▼▼: NaN 값을 포함할 수 있는 per-class accuracy 배열을 안전하게 출력
        def fmt_arr(a):
            if not isinstance(a, list): return str(a)
            return "[" + ", ".join(f"{'nan' if np.isnan(x) else f'{x:.3f}'}" for x in a) + "]"
        # ▲▲▲ NEW/MODIFIED ▲▲▲

        print(
            f"[Epoch {ep:03d}] "
            f"TR loss {tr['loss']:.4f} | VA loss {va['loss']:.4f} | lr {history['lr'][-1]:.2e}\n"
            f"       TR acc/F1 (sev {tr['sev_acc']:.3f} / {tr['sev_macro_f1']:.3f}) | "
            f"VA acc/F1 (sev {va['sev_acc']:.3f} / {va['sev_macro_f1']:.3f})\n"
            f"       TR per-class acc (sev {fmt_arr(tr['sev_cls_acc'])})\n"
            f"       VA per-class acc (sev {fmt_arr(va['sev_cls_acc'])})"
        )
        print()

        current_metric = va["sev_loss"] if sched_mode == "min" else va["sev_macro_f1"]
        is_better = (current_metric < best_val_metric) if sched_mode == "min" else (current_metric > best_val_metric)

        if is_better:
            best_val_metric = current_metric
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {ep} (no improvement for {early_stopping_patience} epochs).")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return history


@torch.no_grad()
def evaluate_on_loader_sev_only( # ▼▼▼ NEW/MODIFIED ▼▼▼: 함수 이름 변경으로 충돌 방지
    model: torch.nn.Module,
    loader,
    device: torch.device,
    *,
    target: str = "severity",
    modality: Optional[str] = None
) -> Dict[str, object]:
    assert target == "severity", "This evaluation function only supports 'severity'."
    assert modality in (None, "acc", "traj"), "modality must be None, 'acc', or 'traj'"

    # 내부 메트릭 계산 함수는 이미 올바르게 구현되어 있음
    def _metrics_from_conf(conf: torch.Tensor):
        return _confmat_to_perclass_acc_macro_f1(conf)

    model.eval()
    epoch_conf = None
    correct = 0.0
    total = 0.0

    pbar = tqdm(loader, desc=f"Eval(severity|{modality or 'both'})", leave=False)
    for batch in pbar:
        acc  = batch["acc_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)
        traj = batch["traj_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)
        mask = batch["mask"].to(device, non_blocking=True)
        y    = batch["y"].to(device, non_blocking=True)

        if modality is None:
            out = model(acc, traj, mask=mask)
        elif modality == "acc":
            out = model(acc,  mask=mask)
        else: # "traj"
            out = model(traj, mask=mask)

        sev_logits, _ = _unpack_sev_only(out)

        preds = sev_logits.argmax(dim=-1)
        y_cpu = y.detach().cpu()
        p_cpu = preds.detach().cpu()

        C = sev_logits.size(-1)
        if epoch_conf is None:
            epoch_conf = torch.zeros(C, C, dtype=torch.long)

        for c in range(C):
            m = (y_cpu == c)
            if m.any():
                binc = torch.bincount(p_cpu[m], minlength=C)
                epoch_conf[c, :] += binc

        correct += (p_cpu == y_cpu).sum().item()
        total   += float(y_cpu.numel())

        mm_now = _metrics_from_conf(epoch_conf)
        acc_now = correct / max(1.0, total)
        pbar.set_postfix({"acc": f"{acc_now:.3f}", "macro_f1": f"{mm_now['macro_f1']:.3f}"})

    if epoch_conf is None:
        return {"acc": 0.0, "macro_f1": 0.0, "per_class_acc": [], "conf": torch.zeros(0, 0)}

    acc_final = correct / max(1.0, total)
    mm = _metrics_from_conf(epoch_conf)
    return {
        "acc": float(acc_final),
        "macro_f1": float(mm["macro_f1"]),
        "per_class_acc": mm["per_class_acc"],
        "conf": epoch_conf,
    }


## Single Modality Train Function

In [ ]:
def _run_epoch_single(
    model: nn.Module,
    loader,
    device: torch.device,
    *,
    is_train: bool,
    target: str,                    # "severity" or "task"
    modality: str,                  # "acc" or "traj"
    criterion: nn.Module = nn.CrossEntropyLoss(),
    optimizer: Optional[torch.optim.Optimizer] = None,
    grad_clip: Optional[float] = 1.0,
    class_weights: Optional[torch.Tensor] = None,
) -> Dict[str, float]:
    assert target in ("severity", "task")
    assert modality in ("acc", "traj")

    model.train(mode=is_train)

    tot_loss = 0.0
    n_batches = 0
    correct = 0.0
    total   = 0.0
    epoch_conf = None  # [C,C] (CPU)

    pbar = tqdm(loader, desc=("Train(single)" if is_train else "Valid(single)"), leave=False)
    for batch in pbar:
        # 입력/전처리
        if modality == "acc":
            x_bag = batch["acc_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)   # [B,N,T,C]
        else:
            x_bag = batch["traj_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)  # [B,N,T,C]

        mask = batch["mask"].to(device, non_blocking=True)  # [B,N] (bool)
        y = (batch["y"] if target == "severity" else batch["y_task"]).to(device, non_blocking=True)

        with torch.set_grad_enabled(is_train):
            logits, _, _ = model(x_bag, mask=mask)  # [B,C]

            if class_weights is not None:
                loss = F.cross_entropy(logits, y, weight=class_weights.to(device=device, dtype=torch.float))
            else:
                loss = criterion(logits, y)

            if is_train:
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                if grad_clip is not None:
                    nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()

        # Metric 누적
        with torch.no_grad():
            preds = logits.argmax(dim=-1)
            correct += (preds == y).sum().item()
            total   += float(y.numel())

            C = logits.size(-1)
            y_cpu = y.detach().cpu()
            p_cpu = preds.detach().cpu()

            if epoch_conf is None:
                epoch_conf = torch.zeros(C, C, dtype=torch.long)

            for c in range(C):
                m = (y_cpu == c)
                if m.any():
                    binc = torch.bincount(p_cpu[m], minlength=C)
                    epoch_conf[c, :] += binc

        tot_loss += float(loss.detach().item())
        n_batches += 1

        acc_now = correct / max(1.0, total)
        mm_now = _confmat_to_perclass_acc_macro_f1(epoch_conf)
        pbar.set_postfix({"loss": f"{tot_loss / n_batches:.4f}",
                          "acc": f"{acc_now:.3f}",
                          "macro_f1": f"{mm_now['macro_f1']:.3f}"})

    if epoch_conf is None:
        return {"loss": 0.0, "acc": 0.0, "macro_f1": 0.0, "per_class_acc": [], "conf": torch.zeros(0, 0)}

    mm_final = _confmat_to_perclass_acc_macro_f1(epoch_conf)
    return {
        "loss": tot_loss / max(1, n_batches),
        "acc": correct / max(1.0, total),
        "macro_f1": mm_final["macro_f1"],
        "per_class_acc": mm_final["per_class_acc"],
        "conf": epoch_conf,
    }


def train_model_single(
    model: nn.Module,
    train_loader,
    valid_loader,
    *,
    device: torch.device,
    modality: str,                 # "acc" or "traj"
    target: str,                   # "severity" or "task"
    epochs: int = 30,
    lr: float = 1e-3,
    weight_decay: float = 1e-4,
    grad_clip: Optional[float] = 1.0,
    early_stopping_patience: int = 10,
    use_scheduler: bool = True,
    sched_mode: str = "min",       # "min": val loss, "max": val macro_f1
    sched_factor: float = 0.5,
    sched_patience: int = 3,
    sched_cooldown: int = 1,
    sched_min_lr: float = 1e-6,
    sched_threshold: float = 1e-3,
    sched_threshold_mode: str = "rel",
    sched_eps: float = 1e-8,
    class_weights: Optional[torch.Tensor] = None,   # size=num_classes
) -> Dict[str, List[float]]:
    assert modality in ("acc", "traj")
    assert target in ("severity", "task")

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    scheduler = None
    if use_scheduler:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=sched_mode,
            factor=sched_factor,
            patience=sched_patience,
            threshold=sched_threshold,
            threshold_mode=sched_threshold_mode,
            cooldown=sched_cooldown,
            min_lr=sched_min_lr,
            eps=sched_eps,
        )

    def _get_lr(opt):
        return opt.param_groups[0]["lr"]

    best_val_score = math.inf if sched_mode == "min" else -math.inf
    best_state = None
    patience = 0

    history = {
        "train_loss": [], "train_acc": [], "train_macro_f1": [], "train_cls_acc": [],
        "valid_loss": [], "valid_acc": [], "valid_macro_f1": [], "valid_cls_acc": [],
        "lr": [],
    }

    for ep in range(1, epochs + 1):
        tr = _run_epoch_single(
            model, train_loader, device,
            is_train=True, target=target, modality=modality,
            optimizer=optimizer, grad_clip=grad_clip, class_weights=class_weights
        )
        va = _run_epoch_single(
            model, valid_loader, device,
            is_train=False, target=target, modality=modality,
            optimizer=None, grad_clip=None, class_weights=class_weights
        )

        prev_lr = _get_lr(optimizer)
        if scheduler is not None:
            monitor_metric = va["loss"] if sched_mode == "min" else va["macro_f1"]
            scheduler.step(monitor_metric)
        cur_lr = _get_lr(optimizer)
        history["lr"].append(cur_lr)

        history["train_loss"].append(tr["loss"]); history["valid_loss"].append(va["loss"])
        history["train_acc"].append(tr["acc"]);   history["valid_acc"].append(va["acc"])
        history["train_macro_f1"].append(tr["macro_f1"]); history["valid_macro_f1"].append(va["macro_f1"])
        history["train_cls_acc"].append(tr["per_class_acc"]); history["valid_cls_acc"].append(va["per_class_acc"])

        def fmt_arr(a):
            return "[" + ", ".join("nan" if (isinstance(x, float) and (x != x)) else f"{float(x):.3f}" for x in a) + "]"
        print(
            f"[Epoch {ep:03d}] "
            f"TR loss {tr['loss']:.4f} | VA loss {va['loss']:.4f} | lr {cur_lr:.2e}\n"
            f"       TR acc {tr['acc']:.3f} | VA acc {va['acc']:.3f}\n"
            f"       TR F1  {tr['macro_f1']:.3f} | VA F1  {va['macro_f1']:.3f}\n"
            f"       TR per-class acc {fmt_arr(tr['per_class_acc'])}\n"
            f"       VA per-class acc {fmt_arr(va['per_class_acc'])}"
        )

        improved = (va["loss"] < best_val_score) if sched_mode == "min" else (va["macro_f1"] > best_val_score)
        if improved:
            best_val_score = va["loss"] if sched_mode == "min" else va["macro_f1"]
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= early_stopping_patience:
                print(f"Early stopping at epoch {ep} (no improvement for {early_stopping_patience} epochs).")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return history

@torch.no_grad()
def evaluate_on_loader_single(
    model: torch.nn.Module,
    loader,
    device: torch.device,
    *,
    target: str = "severity",     # "severity" | "task"
    modality: str = "acc"         # "acc" | "traj"
) -> Dict[str, object]:
    """
    단일 모달(x_bag만 입력) 평가 함수.
    - target: "severity" 또는 "task"
    - modality: "acc" 또는 "traj"
    반환:
      {
        "acc": float,
        "macro_f1": float,
        "per_class_acc": List[float],
        "conf": torch.Tensor [C, C] (CPU)
      }
    """
    assert target in ("severity", "task"), "target must be 'severity' or 'task'"
    assert modality in ("acc", "traj"), "modality must be 'acc' or 'traj'"

    def _metrics_from_conf(conf: torch.Tensor):
        conf = conf.to(torch.float32)
        C = conf.size(0)
        support = conf.sum(dim=1)   # row sum (GT count)
        pred_cnt = conf.sum(dim=0)  # col sum (Pred count)
        tp = torch.diag(conf)

        per_class_acc = torch.full((C,), float('nan'), dtype=torch.float32)
        mask_sup = support > 0
        per_class_acc[mask_sup] = tp[mask_sup] / support[mask_sup]

        precision = torch.zeros(C, dtype=torch.float32)
        recall    = torch.zeros(C, dtype=torch.float32)
        precision[pred_cnt > 0] = tp[pred_cnt > 0] / pred_cnt[pred_cnt > 0]
        recall[mask_sup]        = tp[mask_sup] / support[mask_sup]

        denom = precision + recall
        f1 = torch.zeros(C, dtype=torch.float32)
        nz = denom > 0
        f1[nz] = 2 * precision[nz] * recall[nz] / denom[nz]

        macro_f1 = float(f1[mask_sup].mean().item()) if mask_sup.any() else 0.0
        return {"macro_f1": macro_f1, "per_class_acc": per_class_acc.tolist()}

    model.eval()

    epoch_conf = None  # [C, C] on CPU
    correct = 0.0
    total   = 0.0

    desc = f"Eval(single|{target}|{modality})"
    pbar = tqdm(loader, desc=desc, leave=False)

    for batch in pbar:
        # --- 입력/전처리 ---
        if modality == "acc":
            x_bag = batch["acc_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True)  # [B,N,T,C]
        else:  # "traj"
            x_bag = batch["traj_bag"].permute(0, 1, 3, 2).to(device, non_blocking=True) # [B,N,T,C]

        mask = batch["mask"].to(device, non_blocking=True)  # [B,N] (bool)
        if target == "severity":
            y = batch["y"].to(device, non_blocking=True)
        else:
            y = batch["y_task"].to(device, non_blocking=True)

        # --- 모델 추론 ---
        out = model(x_bag, mask=mask)

        # 다양한 반환 형태를 견고하게 처리
        logits = out[0]

        # --- 메트릭 갱신 ---
        preds = logits.argmax(dim=-1)  # [B]
        y_cpu = y.detach().cpu()
        p_cpu = preds.detach().cpu()

        C = logits.size(-1)
        if epoch_conf is None:
            epoch_conf = torch.zeros(C, C, dtype=torch.long)

        for c in range(C):
            m = (y_cpu == c)
            if m.any():
                binc = torch.bincount(p_cpu[m], minlength=C)
                epoch_conf[c, :] += binc

        correct += (p_cpu == y_cpu).sum().item()
        total   += float(y_cpu.numel())

        # 진행 표시
        mm_now = _metrics_from_conf(epoch_conf)
        acc_now = correct / max(1.0, total)
        pbar.set_postfix({"acc": f"{acc_now:.3f}", "macro_f1": f"{mm_now['macro_f1']:.3f}"})

    if epoch_conf is None:
        return {"acc": 0.0, "macro_f1": 0.0, "per_class_acc": [], "conf": torch.zeros(0, 0)}

    acc_final = correct / max(1.0, total)
    mm = _metrics_from_conf(epoch_conf)

    return {
        "acc": float(acc_final),
        "macro_f1": float(mm["macro_f1"]),
        "per_class_acc": mm["per_class_acc"],
        "conf": epoch_conf,  # CPU tensor
    }

## DataLoader Function

In [ ]:
def build_dataloader(manifest, val_pid, target_per_class=200, filter_task=None, batch_size=16):
    set_seed()

    tr_man, val_man = split_train_valid_by_patient(
        manifest,
        val_pid=[val_pid],
        reset_index=True
    )
    tr_segman, val_segman = expand_train_valid_to_segments(
        tr_man, val_man,
        seg_len=SEG_LEN,
        hop=HOP,
        require_exists=True,
        keep_tail=False,
    )
    bag_rot = BagRotator(seed=42, prob=1.0)
    inst_ptw = InstancePermTimeW(seed=42, n_min=1, n_max=5,
                                tw_a=(0.15,0.35), tw_f=(0.5,1.5),
                                apply_perm=True, apply_timew=True,
                                prob_perm=1.0, prob_timew=1.0)


    # ② 클래스당 개수로 직접 지정(예: 각 클래스 180개)
    mil_train = BalancedMILDatasetPath_RPT(
        tr_segman,
        bag_rot=bag_rot, inst_perm_timew=inst_ptw,
        max_instances_per_bag=10,
        target_per_class=target_per_class,       # ← 추가
        allow_downsample=False,
        print_stats=True,
        filter_task=filter_task
    )

    mil_valid = EvalMILDatasetPath(val_segman, target_col="target", filter_task=filter_task)

    dl_tr = DataLoader(mil_train, batch_size=batch_size, shuffle=True,  num_workers=NUM_WORKERS,
                    pin_memory=True, collate_fn=lambda b: collate_mil_pad(b, concat_channels=False))
    dl_va = DataLoader(mil_valid,     batch_size=batch_size, shuffle=False, num_workers=NUM_WORKERS,
                    pin_memory=True, collate_fn=lambda b: collate_mil_pad(b, concat_channels=False))
    return dl_tr, dl_va

def build_LOSO_loaders(manifest, target_per_class=200, filter_task=None):
    loaders = {}
    for i in range(1, 10):
        dl_tr, dl_va = build_dataloader(manifest, str(i), target_per_class=target_per_class, filter_task=filter_task, batch_size=16)
        loaders[str(i)] = (dl_tr, dl_va)
    return loaders

In [ ]:
manifest = build_manifest(LABEL_DIR, ROOT, target_col='target_k5')
# manifest = build_manifest(LABEL_DIR, ROOT, target_col='tetras_score')
loaders = build_LOSO_loaders(manifest)

Expand to segments: 100%|██████████| 41/41 [00:00<00:00, 99.48file/s]


[MIL Balance (R+P+T) @ path-bag]
  Task Filter: None
  before : {0: 37, 1: 65, 2: 112, 3: 43}
  desired: {0: 200, 1: 200, 2: 200, 3: 200}
  after  : {0: 200, 1: 200, 2: 200, 3: 200} | total: 800


Expand to segments: 100%|██████████| 25/25 [00:00<00:00, 97.19file/s] 


[MIL Balance (R+P+T) @ path-bag]
  Task Filter: None
  before : {0: 56, 1: 67, 2: 109, 3: 41}
  desired: {0: 200, 1: 200, 2: 200, 3: 200}
  after  : {0: 200, 1: 200, 2: 200, 3: 200} | total: 800


Expand to segments: 100%|██████████| 61/61 [00:00<00:00, 144.00file/s]


...

[MIL Balance (R+P+T) @ path-bag]
  Task Filter: None
  before : {0: 57, 1: 68, 2: 118, 3: 43}
  desired: {0: 200, 1: 200, 2: 200, 3: 200}
  after  : {0: 200, 1: 200, 2: 200, 3: 200} | total: 800


Expand to segments: 100%|██████████| 30/30 [00:00<00:00, 96.83file/s]


[MIL Balance (R+P+T) @ path-bag]
  Task Filter: None
  before : {0: 62, 1: 67, 2: 96, 3: 43}
  desired: {0: 200, 1: 200, 2: 200, 3: 200}
  after  : {0: 200, 1: 200, 2: 200, 3: 200} | total: 800


Expand to segments: 100%|██████████| 36/36 [00:00<00:00, 131.82file/s]


[MIL Balance (R+P+T) @ path-bag]
  Task Filter: None
  before : {0: 57, 1: 62, 2: 103, 3: 40}
  desired: {0: 200, 1: 200, 2: 200, 3: 200}
  after  : {0: 200, 1: 200, 2: 200, 3: 200} | total: 800


# Train

## Multimodal

#### ACC encoder : LSTM

In [ ]:
acc_name = 'LSTM'
traj_name = 'LSTM'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4327 | VA loss 1.4822 | lr 1.00e-03
       TR acc/F1 (sev 0.250 / 0.244) | VA acc/F1 (sev 0.000 / 0.000)
       TR per-class acc (sev [0.295, 0.175, 0.165, 0.365])
       VA per-class acc (sev [0.000, 0.000, 0.000, nan])



[Epoch 002] TR loss 1.4021 | VA loss 1.2360 | lr 1.00e-03
       TR acc/F1 (sev 0.264 / 0.256) | VA acc/F1 (sev 0.244 / 0.131)
       TR per-class acc (sev [0.195, 0.150, 0.365, 0.345])
       VA per-class acc (sev [0.000, 1.000, 0.000, nan])



[Epoch 003] TR loss 1.3963 | VA loss 1.2906 | lr 1.00e-03
       TR acc/F1 (sev 0.294 / 0.287) | VA acc/F1 (sev 0.463 / 0.258)
       TR per-class acc (sev [0.150, 0.280, 0.325, 0.420])
       VA per-class acc (sev [0.720, 0.100, 0.000, nan])



[Epoch 004] TR loss 1.3550 | VA loss 1.2298 | lr 1.00e-03
       TR acc/F1 (sev 0.304 / 0.297) | VA acc/F1 (sev 0.488 / 0.222)
       TR per-class acc (sev [0.185, 0.235, 0.335, 0.460])
       VA per-class acc (sev [0.800, 0.000, 0.000, nan])



...

[Epoch 055] TR loss 0.4638 | VA loss 0.6737 | lr 6.25e-05
       TR acc/F1 (sev 0.794 / 0.794) | VA acc/F1 (sev 0.611 / 0.517)
       TR per-class acc (sev [0.905, 0.660, 0.720, 0.890])
       VA per-class acc (sev [0.200, 0.846, 0.533, 0.667])



[Epoch 056] TR loss 0.5267 | VA loss 1.0492 | lr 6.25e-05
       TR acc/F1 (sev 0.795 / 0.795) | VA acc/F1 (sev 0.528 / 0.397)
       TR per-class acc (sev [0.885, 0.670, 0.720, 0.905])
       VA per-class acc (sev [0.000, 0.846, 0.333, 1.000])



[Epoch 057] TR loss 0.4813 | VA loss 0.6191 | lr 6.25e-05
       TR acc/F1 (sev 0.801 / 0.801) | VA acc/F1 (sev 0.611 / 0.511)
       TR per-class acc (sev [0.870, 0.660, 0.765, 0.910])
       VA per-class acc (sev [0.200, 0.846, 0.533, 0.667])



[Epoch 058] TR loss 0.4638 | VA loss 0.8055 | lr 6.25e-05
       TR acc/F1 (sev 0.799 / 0.798) | VA acc/F1 (sev 0.611 / 0.465)
       TR per-class acc (sev [0.910, 0.660, 0.715, 0.910])
       VA per-class acc (sev [0.000, 0.846, 0.533, 1.000])

Early stopping at epoch 58 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8333 | macro-F1=0.7844 | per-class acc=[0.6, 0.769, 1.0, 0.667]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.9512 | macro-F1=0.9111
PID= 2 | ACC=0.6800 | macro-F1=0.5580
PID= 3 | ACC=0.6885 | macro-F1=0.5181
PID= 4 | ACC=0.7419 | macro-F1=0.3444
PID= 5 | ACC=0.7273 | macro-F1=0.4732
PID= 6 | ACC=0.8889 | macro-F1=0.6000
PID= 7 | ACC=0.7500 | macro-F1=0.7483
PID= 8 | ACC=0.8667 | macro-F1=0.8295
PID= 9 | ACC=0.8333 | macro-F1=0.7844
-----------------------------------------------
ACC   mean±std: 0.7920 ± 0.0960
F1    mean±std: 0.6408 ± 0.1872


In [ ]:
acc_name = 'LSTM'
traj_name = 'ResNet18'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )
    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4291 | VA loss 1.3807 | lr 1.00e-03
       TR acc/F1 (sev 0.241 / 0.238) | VA acc/F1 (sev 0.244 / 0.131)
       TR per-class acc (sev [0.330, 0.255, 0.145, 0.235])
       VA per-class acc (sev [0.000, 1.000, 0.000, nan])



[Epoch 002] TR loss 1.3552 | VA loss 1.3061 | lr 1.00e-03
       TR acc/F1 (sev 0.350 / 0.344) | VA acc/F1 (sev 0.122 / 0.093)
       TR per-class acc (sev [0.190, 0.385, 0.335, 0.490])
       VA per-class acc (sev [0.000, 0.500, 0.000, nan])



[Epoch 003] TR loss 1.2758 | VA loss 1.4531 | lr 1.00e-03
       TR acc/F1 (sev 0.390 / 0.354) | VA acc/F1 (sev 0.122 / 0.211)
       TR per-class acc (sev [0.230, 0.085, 0.540, 0.705])
       VA per-class acc (sev [0.000, 0.100, 0.667, nan])



[Epoch 004] TR loss 1.2887 | VA loss 1.3180 | lr 1.00e-03
       TR acc/F1 (sev 0.398 / 0.367) | VA acc/F1 (sev 0.317 / 0.260)
       TR per-class acc (sev [0.350, 0.105, 0.440, 0.695])
       VA per-class acc (sev [0.400, 0.300, 0.000, nan])



...

[Epoch 022] TR loss 1.0750 | VA loss 1.2826 | lr 2.50e-04
       TR acc/F1 (sev 0.481 / 0.468) | VA acc/F1 (sev 0.139 / 0.076)
       TR per-class acc (sev [0.535, 0.220, 0.380, 0.790])
       VA per-class acc (sev [1.000, 0.000, 0.000, 0.000])



[Epoch 023] TR loss 1.1279 | VA loss 1.2451 | lr 2.50e-04
       TR acc/F1 (sev 0.472 / 0.440) | VA acc/F1 (sev 0.250 / 0.252)
       TR per-class acc (sev [0.650, 0.180, 0.255, 0.805])
       VA per-class acc (sev [1.000, 0.000, 0.133, 0.667])



[Epoch 024] TR loss 1.0468 | VA loss 1.2794 | lr 2.50e-04
       TR acc/F1 (sev 0.482 / 0.456) | VA acc/F1 (sev 0.139 / 0.081)
       TR per-class acc (sev [0.590, 0.185, 0.300, 0.855])
       VA per-class acc (sev [1.000, 0.000, 0.000, 0.000])



[Epoch 025] TR loss 1.0855 | VA loss 1.2387 | lr 2.50e-04
       TR acc/F1 (sev 0.436 / 0.426) | VA acc/F1 (sev 0.389 / 0.343)
       TR per-class acc (sev [0.450, 0.240, 0.270, 0.785])
       VA per-class acc (sev [1.000, 0.000, 0.467, 0.667])

Early stopping at epoch 25 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.3056 | macro-F1=0.2507 | per-class acc=[0.2, 0.0, 0.533, 0.667]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.7317 | macro-F1=0.4731
PID= 2 | ACC=0.4800 | macro-F1=0.3137
PID= 3 | ACC=0.4590 | macro-F1=0.1707
PID= 4 | ACC=0.5484 | macro-F1=0.2150
PID= 5 | ACC=0.7727 | macro-F1=0.5706
PID= 6 | ACC=0.6667 | macro-F1=0.2667
PID= 7 | ACC=0.8333 | macro-F1=0.8286
PID= 8 | ACC=0.4667 | macro-F1=0.3333
PID= 9 | ACC=0.3056 | macro-F1=0.2507
-----------------------------------------------
ACC   mean±std: 0.5849 ± 0.1752
F1    mean±std: 0.3803 ± 0.2102


In [ ]:
acc_name = 'LSTM'
traj_name = 'TimesNet'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )
    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

[Fold PID=1]


[Epoch 001] TR loss 1.4304 | VA loss 1.3629 | lr 1.00e-03
       TR acc/F1 (sev 0.244 / 0.240) | VA acc/F1 (sev 0.488 / 0.219)
       TR per-class acc (sev [0.160, 0.245, 0.230, 0.340])
       VA per-class acc (sev [0.800, 0.000, 0.000, nan])



[Epoch 002] TR loss 1.3994 | VA loss 1.4762 | lr 1.00e-03
       TR acc/F1 (sev 0.256 / 0.254) | VA acc/F1 (sev 0.000 / 0.000)
       TR per-class acc (sev [0.335, 0.200, 0.235, 0.255])
       VA per-class acc (sev [0.000, 0.000, 0.000, nan])



[Epoch 003] TR loss 1.3905 | VA loss 1.5266 | lr 1.00e-03
       TR acc/F1 (sev 0.279 / 0.266) | VA acc/F1 (sev 0.220 / 0.140)
       TR per-class acc (sev [0.095, 0.320, 0.400, 0.300])
       VA per-class acc (sev [0.000, 0.900, 0.000, nan])



[Epoch 004] TR loss 1.3749 | VA loss 1.3814 | lr 1.00e-03
       TR acc/F1 (sev 0.290 / 0.280) | VA acc/F1 (sev 0.244 / 0.148)
       TR per-class acc (sev [0.125, 0.265, 0.375, 0.395])
       VA per-class acc (sev [0.400, 0.000, 0.000, nan])



...

[Epoch 062] TR loss 0.5291 | VA loss 0.7851 | lr 3.13e-05
       TR acc/F1 (sev 0.776 / 0.776) | VA acc/F1 (sev 0.750 / 0.661)
       TR per-class acc (sev [0.875, 0.665, 0.700, 0.865])
       VA per-class acc (sev [0.400, 0.923, 0.733, 0.667])



[Epoch 063] TR loss 0.5106 | VA loss 0.7538 | lr 3.13e-05
       TR acc/F1 (sev 0.781 / 0.781) | VA acc/F1 (sev 0.750 / 0.661)
       TR per-class acc (sev [0.885, 0.665, 0.690, 0.885])
       VA per-class acc (sev [0.400, 0.923, 0.733, 0.667])



[Epoch 064] TR loss 0.5269 | VA loss 0.7866 | lr 3.13e-05
       TR acc/F1 (sev 0.777 / 0.775) | VA acc/F1 (sev 0.778 / 0.689)
       TR per-class acc (sev [0.880, 0.595, 0.735, 0.900])
       VA per-class acc (sev [0.400, 0.923, 0.800, 0.667])



[Epoch 065] TR loss 0.5337 | VA loss 0.7654 | lr 3.13e-05
       TR acc/F1 (sev 0.802 / 0.803) | VA acc/F1 (sev 0.750 / 0.661)
       TR per-class acc (sev [0.875, 0.685, 0.745, 0.905])
       VA per-class acc (sev [0.400, 0.923, 0.733, 0.667])

Early stopping at epoch 65 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.7222 | macro-F1=0.6423 | per-class acc=[0.4, 0.923, 0.667, 0.667]


In [ ]:
acc_name = 'LSTM'
traj_name = 'MyWaveNet'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4254 | VA loss 1.3958 | lr 1.00e-03
       TR acc/F1 (sev 0.289 / 0.289) | VA acc/F1 (sev 0.244 / 0.131)
       TR per-class acc (sev [0.305, 0.250, 0.290, 0.310])
       VA per-class acc (sev [0.000, 1.000, 0.000, nan])



[Epoch 002] TR loss 1.4077 | VA loss 1.3740 | lr 1.00e-03
       TR acc/F1 (sev 0.254 / 0.240) | VA acc/F1 (sev 0.171 / 0.114)
       TR per-class acc (sev [0.175, 0.110, 0.340, 0.390])
       VA per-class acc (sev [0.040, 0.000, 1.000, nan])



[Epoch 003] TR loss 1.3781 | VA loss 1.4316 | lr 1.00e-03
       TR acc/F1 (sev 0.279 / 0.277) | VA acc/F1 (sev 0.220 / 0.122)
       TR per-class acc (sev [0.265, 0.225, 0.250, 0.375])
       VA per-class acc (sev [0.000, 0.900, 0.000, nan])



[Epoch 004] TR loss 1.3936 | VA loss 1.4352 | lr 1.00e-03
       TR acc/F1 (sev 0.297 / 0.273) | VA acc/F1 (sev 0.146 / 0.098)
       TR per-class acc (sev [0.115, 0.370, 0.160, 0.545])
       VA per-class acc (sev [0.000, 0.600, 0.000, nan])



...

[Epoch 079] TR loss 0.3637 | VA loss 0.9587 | lr 1.25e-04
       TR acc/F1 (sev 0.849 / 0.848) | VA acc/F1 (sev 0.722 / 0.637)
       TR per-class acc (sev [0.945, 0.730, 0.770, 0.950])
       VA per-class acc (sev [1.000, 0.385, 1.000, 0.333])



[Epoch 080] TR loss 0.3543 | VA loss 1.0900 | lr 1.25e-04
       TR acc/F1 (sev 0.859 / 0.857) | VA acc/F1 (sev 0.694 / 0.505)
       TR per-class acc (sev [0.945, 0.735, 0.785, 0.970])
       VA per-class acc (sev [1.000, 0.385, 1.000, 0.000])



[Epoch 081] TR loss 0.3355 | VA loss 1.0011 | lr 1.25e-04
       TR acc/F1 (sev 0.856 / 0.856) | VA acc/F1 (sev 0.722 / 0.637)
       TR per-class acc (sev [0.935, 0.730, 0.795, 0.965])
       VA per-class acc (sev [1.000, 0.385, 1.000, 0.333])



[Epoch 082] TR loss 0.3490 | VA loss 1.0329 | lr 1.25e-04
       TR acc/F1 (sev 0.860 / 0.860) | VA acc/F1 (sev 0.722 / 0.532)
       TR per-class acc (sev [0.940, 0.765, 0.795, 0.940])
       VA per-class acc (sev [1.000, 0.462, 1.000, 0.000])

Early stopping at epoch 82 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8889 | macro-F1=0.8438 | per-class acc=[0.8, 0.846, 1.0, 0.667]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.9024 | macro-F1=0.8575
PID= 2 | ACC=1.0000 | macro-F1=1.0000
PID= 3 | ACC=0.7049 | macro-F1=0.6364
PID= 4 | ACC=0.7581 | macro-F1=0.3335
PID= 5 | ACC=0.7273 | macro-F1=0.4830
PID= 6 | ACC=0.6667 | macro-F1=0.4111
PID= 7 | ACC=1.0000 | macro-F1=1.0000
PID= 8 | ACC=0.9000 | macro-F1=0.8952
PID= 9 | ACC=0.8889 | macro-F1=0.8438
-----------------------------------------------
ACC   mean±std: 0.8387 ± 0.1269
F1    mean±std: 0.7178 ± 0.2573


#### ACC encoder : ResNet18

In [ ]:
acc_name = 'ResNet18'
traj_name = 'LSTM'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.3602 | VA loss 1.2659 | lr 1.00e-03
       TR acc/F1 (sev 0.385 / 0.381) | VA acc/F1 (sev 0.439 / 0.318)
       TR per-class acc (sev [0.370, 0.265, 0.390, 0.515])
       VA per-class acc (sev [0.360, 0.900, 0.000, nan])



[Epoch 002] TR loss 1.2366 | VA loss 1.1642 | lr 1.00e-03
       TR acc/F1 (sev 0.435 / 0.414) | VA acc/F1 (sev 0.366 / 0.280)
       TR per-class acc (sev [0.480, 0.215, 0.290, 0.755])
       VA per-class acc (sev [0.520, 0.200, 0.000, nan])



[Epoch 003] TR loss 1.2063 | VA loss 1.0908 | lr 1.00e-03
       TR acc/F1 (sev 0.429 / 0.408) | VA acc/F1 (sev 0.512 / 0.303)
       TR per-class acc (sev [0.495, 0.165, 0.305, 0.750])
       VA per-class acc (sev [0.800, 0.000, 0.167, nan])



[Epoch 004] TR loss 1.0427 | VA loss 0.7898 | lr 1.00e-03
       TR acc/F1 (sev 0.534 / 0.493) | VA acc/F1 (sev 0.659 / 0.431)
       TR per-class acc (sev [0.715, 0.110, 0.480, 0.830])
       VA per-class acc (sev [1.000, 0.000, 0.333, nan])



...

[Epoch 035] TR loss 0.4678 | VA loss 1.6352 | lr 2.50e-04
       TR acc/F1 (sev 0.812 / 0.813) | VA acc/F1 (sev 0.389 / 0.314)
       TR per-class acc (sev [0.870, 0.660, 0.805, 0.915])
       VA per-class acc (sev [1.000, 0.308, 0.333, 0.000])



[Epoch 036] TR loss 0.4490 | VA loss 2.0221 | lr 2.50e-04
       TR acc/F1 (sev 0.820 / 0.819) | VA acc/F1 (sev 0.417 / 0.334)
       TR per-class acc (sev [0.890, 0.715, 0.750, 0.925])
       VA per-class acc (sev [1.000, 0.308, 0.400, 0.000])



[Epoch 037] TR loss 0.3897 | VA loss 2.1454 | lr 2.50e-04
       TR acc/F1 (sev 0.834 / 0.833) | VA acc/F1 (sev 0.333 / 0.272)
       TR per-class acc (sev [0.915, 0.750, 0.740, 0.930])
       VA per-class acc (sev [1.000, 0.308, 0.200, 0.000])



[Epoch 038] TR loss 0.3968 | VA loss 1.6839 | lr 2.50e-04
       TR acc/F1 (sev 0.835 / 0.834) | VA acc/F1 (sev 0.472 / 0.369)
       TR per-class acc (sev [0.925, 0.775, 0.715, 0.925])
       VA per-class acc (sev [1.000, 0.308, 0.533, 0.000])

Early stopping at epoch 38 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.6944 | macro-F1=0.4935 | per-class acc=[0.6, 0.538, 1.0, 0.0]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.8293 | macro-F1=0.7383
PID= 2 | ACC=0.8000 | macro-F1=0.6530
PID= 3 | ACC=0.7869 | macro-F1=0.6649
PID= 4 | ACC=0.8226 | macro-F1=0.6238
PID= 5 | ACC=0.8182 | macro-F1=0.8125
PID= 6 | ACC=0.8889 | macro-F1=0.8632
PID= 7 | ACC=0.8333 | macro-F1=0.8286
PID= 8 | ACC=0.9000 | macro-F1=0.8667
PID= 9 | ACC=0.6944 | macro-F1=0.4935
-----------------------------------------------
ACC   mean±std: 0.8193 ± 0.0599
F1    mean±std: 0.7272 ± 0.1276


In [ ]:
acc_name = 'ResNet18'
traj_name = 'ResNet18'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.3976 | VA loss 1.7154 | lr 1.00e-03
       TR acc/F1 (sev 0.318 / 0.311) | VA acc/F1 (sev 0.122 / 0.167)
       TR per-class acc (sev [0.255, 0.195, 0.355, 0.465])
       VA per-class acc (sev [0.000, 0.100, 0.667, nan])



[Epoch 002] TR loss 1.2706 | VA loss 1.1793 | lr 1.00e-03
       TR acc/F1 (sev 0.404 / 0.391) | VA acc/F1 (sev 0.390 / 0.201)
       TR per-class acc (sev [0.425, 0.160, 0.435, 0.595])
       VA per-class acc (sev [0.640, 0.000, 0.000, nan])



[Epoch 003] TR loss 1.1167 | VA loss 1.0546 | lr 1.00e-03
       TR acc/F1 (sev 0.495 / 0.485) | VA acc/F1 (sev 0.634 / 0.456)
       TR per-class acc (sev [0.540, 0.250, 0.435, 0.755])
       VA per-class acc (sev [0.920, 0.000, 0.500, nan])



[Epoch 004] TR loss 1.0833 | VA loss 0.9101 | lr 1.00e-03
       TR acc/F1 (sev 0.507 / 0.478) | VA acc/F1 (sev 0.659 / 0.530)
       TR per-class acc (sev [0.620, 0.135, 0.450, 0.825])
       VA per-class acc (sev [0.920, 0.100, 0.500, nan])



...

[Epoch 039] TR loss 0.5398 | VA loss 1.3242 | lr 1.25e-04
       TR acc/F1 (sev 0.764 / 0.759) | VA acc/F1 (sev 0.528 / 0.390)
       TR per-class acc (sev [0.885, 0.575, 0.685, 0.910])
       VA per-class acc (sev [1.000, 0.231, 0.733, 0.000])



[Epoch 040] TR loss 0.5348 | VA loss 1.1249 | lr 1.25e-04
       TR acc/F1 (sev 0.779 / 0.774) | VA acc/F1 (sev 0.556 / 0.416)
       TR per-class acc (sev [0.875, 0.560, 0.790, 0.890])
       VA per-class acc (sev [1.000, 0.308, 0.733, 0.000])



[Epoch 041] TR loss 0.5529 | VA loss 0.9818 | lr 1.25e-04
       TR acc/F1 (sev 0.776 / 0.776) | VA acc/F1 (sev 0.639 / 0.474)
       TR per-class acc (sev [0.860, 0.650, 0.715, 0.880])
       VA per-class acc (sev [1.000, 0.385, 0.867, 0.000])



[Epoch 042] TR loss 0.5719 | VA loss 0.6580 | lr 1.25e-04
       TR acc/F1 (sev 0.766 / 0.764) | VA acc/F1 (sev 0.722 / 0.527)
       TR per-class acc (sev [0.845, 0.585, 0.755, 0.880])
       VA per-class acc (sev [0.800, 0.538, 1.000, 0.000])

Early stopping at epoch 42 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8889 | macro-F1=0.6902 | per-class acc=[0.8, 1.0, 1.0, 0.0]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.8780 | macro-F1=0.8796
PID= 2 | ACC=0.9200 | macro-F1=0.7250
PID= 3 | ACC=0.6885 | macro-F1=0.6364
PID= 4 | ACC=0.7742 | macro-F1=0.5974
PID= 5 | ACC=0.8636 | macro-F1=0.7933
PID= 6 | ACC=1.0000 | macro-F1=1.0000
PID= 7 | ACC=0.9167 | macro-F1=0.9111
PID= 8 | ACC=0.8333 | macro-F1=0.8053
PID= 9 | ACC=0.8889 | macro-F1=0.6902
-----------------------------------------------
ACC   mean±std: 0.8626 ± 0.0901
F1    mean±std: 0.7820 ± 0.1329


In [ ]:
acc_name = 'ResNet18'
traj_name = 'TimesNet'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
acc_name = 'ResNet18'
traj_name = 'MyWaveNet'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

#### ACC encoder : TImesNet

#### ACC encoder : MyWaveNet

In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'LSTM'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4316 | VA loss 1.5834 | lr 1.00e-03
       TR acc/F1 (sev 0.280 / 0.280) | VA acc/F1 (sev 0.000 / 0.000)
       TR per-class acc (sev [0.245, 0.265, 0.280, 0.330])
       VA per-class acc (sev [0.000, 0.000, 0.000, nan])



[Epoch 002] TR loss 1.4054 | VA loss 1.3321 | lr 1.00e-03
       TR acc/F1 (sev 0.284 / 0.280) | VA acc/F1 (sev 0.244 / 0.131)
       TR per-class acc (sev [0.280, 0.255, 0.210, 0.390])
       VA per-class acc (sev [0.000, 1.000, 0.000, nan])



[Epoch 003] TR loss 1.3820 | VA loss 1.4283 | lr 1.00e-03
       TR acc/F1 (sev 0.270 / 0.260) | VA acc/F1 (sev 0.220 / 0.235)
       TR per-class acc (sev [0.115, 0.250, 0.310, 0.405])
       VA per-class acc (sev [0.160, 0.500, 0.000, nan])



[Epoch 004] TR loss 1.3355 | VA loss 1.3544 | lr 1.00e-03
       TR acc/F1 (sev 0.344 / 0.330) | VA acc/F1 (sev 0.098 / 0.112)
       TR per-class acc (sev [0.155, 0.340, 0.325, 0.555])
       VA per-class acc (sev [0.000, 0.100, 0.500, nan])



...

[Epoch 083] TR loss 0.1360 | VA loss 0.0852 | lr 6.25e-05
       TR acc/F1 (sev 0.949 / 0.949) | VA acc/F1 (sev 1.000 / 1.000)
       TR per-class acc (sev [0.975, 0.920, 0.935, 0.965])
       VA per-class acc (sev [nan, 1.000, 1.000, nan])



[Epoch 084] TR loss 0.1238 | VA loss 0.2658 | lr 6.25e-05
       TR acc/F1 (sev 0.946 / 0.946) | VA acc/F1 (sev 0.867 / 0.841)
       TR per-class acc (sev [0.975, 0.905, 0.940, 0.965])
       VA per-class acc (sev [nan, 0.625, 0.955, nan])



[Epoch 085] TR loss 0.1506 | VA loss 0.0738 | lr 6.25e-05
       TR acc/F1 (sev 0.939 / 0.939) | VA acc/F1 (sev 1.000 / 1.000)
       TR per-class acc (sev [0.955, 0.905, 0.945, 0.950])
       VA per-class acc (sev [nan, 1.000, 1.000, nan])



[Epoch 086] TR loss 0.1328 | VA loss 0.1202 | lr 6.25e-05
       TR acc/F1 (sev 0.950 / 0.950) | VA acc/F1 (sev 0.933 / 0.907)
       TR per-class acc (sev [0.975, 0.940, 0.925, 0.960])
       VA per-class acc (sev [nan, 0.750, 1.000, nan])



Train:  34%|███▍      | 17/50 [00:07<00:12,  2.71it/s, sev_loss=0.1557, sev_acc=0.938, sev_f1=0.939]

In [ ]:
# 8, 9 만
acc_name = 'MyWaveNet'
traj_name = 'LSTM'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    if patient_id not in ['8', '9']:
        continue
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

[Fold PID=8]


[Epoch 001] TR loss 1.4475 | VA loss 1.4457 | lr 1.00e-03
       TR acc/F1 (sev 0.231 / 0.226) | VA acc/F1 (sev 0.267 / 0.211)
       TR per-class acc (sev [0.230, 0.300, 0.125, 0.270])
       VA per-class acc (sev [nan, 1.000, 0.000, nan])



[Epoch 002] TR loss 1.4027 | VA loss 1.5166 | lr 1.00e-03
       TR acc/F1 (sev 0.250 / 0.245) | VA acc/F1 (sev 0.267 / 0.211)
       TR per-class acc (sev [0.215, 0.155, 0.335, 0.295])
       VA per-class acc (sev [nan, 1.000, 0.000, nan])



[Epoch 003] TR loss 1.3715 | VA loss 1.4633 | lr 1.00e-03
       TR acc/F1 (sev 0.300 / 0.300) | VA acc/F1 (sev 0.000 / 0.000)
       TR per-class acc (sev [0.235, 0.325, 0.280, 0.360])
       VA per-class acc (sev [nan, 0.000, 0.000, nan])



[Epoch 004] TR loss 1.2871 | VA loss 1.4343 | lr 1.00e-03
       TR acc/F1 (sev 0.366 / 0.367) | VA acc/F1 (sev 0.000 / 0.000)
       TR per-class acc (sev [0.310, 0.295, 0.335, 0.525])
       VA per-class acc (sev [nan, 0.000, 0.000, nan])



...

[Epoch 052] TR loss 0.3048 | VA loss 1.0156 | lr 3.13e-05
       TR acc/F1 (sev 0.885 / 0.885) | VA acc/F1 (sev 0.583 / 0.591)
       TR per-class acc (sev [0.955, 0.825, 0.835, 0.925])
       VA per-class acc (sev [1.000, 0.231, 0.667, 1.000])



[Epoch 053] TR loss 0.2971 | VA loss 1.1609 | lr 3.13e-05
       TR acc/F1 (sev 0.886 / 0.885) | VA acc/F1 (sev 0.556 / 0.557)
       TR per-class acc (sev [0.970, 0.810, 0.820, 0.945])
       VA per-class acc (sev [1.000, 0.077, 0.800, 0.667])



[Epoch 054] TR loss 0.3084 | VA loss 0.7872 | lr 3.13e-05
       TR acc/F1 (sev 0.896 / 0.896) | VA acc/F1 (sev 0.639 / 0.638)
       TR per-class acc (sev [0.975, 0.830, 0.850, 0.930])
       VA per-class acc (sev [1.000, 0.462, 0.600, 1.000])



[Epoch 055] TR loss 0.2915 | VA loss 0.8691 | lr 3.13e-05
       TR acc/F1 (sev 0.886 / 0.886) | VA acc/F1 (sev 0.611 / 0.612)
       TR per-class acc (sev [0.950, 0.785, 0.875, 0.935])
       VA per-class acc (sev [1.000, 0.385, 0.600, 1.000])

Early stopping at epoch 55 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8333 | macro-F1=0.7923 | per-class acc=[0.8, 0.923, 0.733, 1.0]


In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'ResNet18'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4610 | VA loss 1.2795 | lr 1.00e-03
       TR acc/F1 (sev 0.260 / 0.253) | VA acc/F1 (sev 0.293 / 0.216)
       TR per-class acc (sev [0.140, 0.315, 0.230, 0.355])
       VA per-class acc (sev [0.320, 0.000, 0.667, nan])



[Epoch 002] TR loss 1.3982 | VA loss 1.4553 | lr 1.00e-03
       TR acc/F1 (sev 0.278 / 0.271) | VA acc/F1 (sev 0.146 / 0.085)
       TR per-class acc (sev [0.255, 0.185, 0.235, 0.435])
       VA per-class acc (sev [0.000, 0.000, 1.000, nan])



[Epoch 003] TR loss 1.3657 | VA loss 1.2555 | lr 1.00e-03
       TR acc/F1 (sev 0.325 / 0.281) | VA acc/F1 (sev 0.585 / 0.246)
       TR per-class acc (sev [0.055, 0.190, 0.345, 0.710])
       VA per-class acc (sev [0.960, 0.000, 0.000, nan])



[Epoch 004] TR loss 1.3085 | VA loss 1.2422 | lr 1.00e-03
       TR acc/F1 (sev 0.365 / 0.332) | VA acc/F1 (sev 0.195 / 0.128)
       TR per-class acc (sev [0.475, 0.120, 0.185, 0.680])
       VA per-class acc (sev [0.040, 0.700, 0.000, nan])



...

[Epoch 052] TR loss 0.8560 | VA loss 1.0892 | lr 3.13e-05
       TR acc/F1 (sev 0.588 / 0.578) | VA acc/F1 (sev 0.444 / 0.408)
       TR per-class acc (sev [0.695, 0.410, 0.370, 0.875])
       VA per-class acc (sev [1.000, 0.077, 0.600, 0.333])



[Epoch 053] TR loss 0.8705 | VA loss 1.0517 | lr 3.13e-05
       TR acc/F1 (sev 0.609 / 0.599) | VA acc/F1 (sev 0.611 / 0.531)
       TR per-class acc (sev [0.760, 0.375, 0.435, 0.865])
       VA per-class acc (sev [1.000, 0.154, 0.933, 0.333])



[Epoch 054] TR loss 0.8682 | VA loss 1.0602 | lr 3.13e-05
       TR acc/F1 (sev 0.598 / 0.590) | VA acc/F1 (sev 0.556 / 0.462)
       TR per-class acc (sev [0.700, 0.355, 0.460, 0.875])
       VA per-class acc (sev [1.000, 0.077, 0.867, 0.333])



[Epoch 055] TR loss 0.8744 | VA loss 1.0346 | lr 3.13e-05
       TR acc/F1 (sev 0.610 / 0.603) | VA acc/F1 (sev 0.417 / 0.418)
       TR per-class acc (sev [0.700, 0.435, 0.410, 0.895])
       VA per-class acc (sev [1.000, 0.231, 0.400, 0.333])

Early stopping at epoch 55 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.5556 | macro-F1=0.4708 | per-class acc=[1.0, 0.154, 0.8, 0.333]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.9268 | macro-F1=0.9042
PID= 2 | ACC=1.0000 | macro-F1=1.0000
PID= 3 | ACC=0.7705 | macro-F1=0.6629
PID= 4 | ACC=0.8065 | macro-F1=0.8028
PID= 5 | ACC=0.9091 | macro-F1=0.6886
PID= 6 | ACC=0.8889 | macro-F1=0.9030
PID= 7 | ACC=0.9167 | macro-F1=0.9111
PID= 8 | ACC=0.9667 | macro-F1=0.9556
PID= 9 | ACC=0.5556 | macro-F1=0.4708
-----------------------------------------------
ACC   mean±std: 0.8601 ± 0.1348
F1    mean±std: 0.8110 ± 0.1719


In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'TimesNet'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4544 | VA loss 1.3566 | lr 1.00e-03
       TR acc/F1 (sev 0.266 / 0.262) | VA acc/F1 (sev 0.146 / 0.085)
       TR per-class acc (sev [0.255, 0.380, 0.260, 0.170])
       VA per-class acc (sev [0.000, 0.000, 1.000, nan])



[Epoch 002] TR loss 1.3979 | VA loss 1.5052 | lr 1.00e-03
       TR acc/F1 (sev 0.264 / 0.246) | VA acc/F1 (sev 0.073 / 0.056)
       TR per-class acc (sev [0.075, 0.355, 0.195, 0.430])
       VA per-class acc (sev [0.000, 0.300, 0.000, nan])



[Epoch 003] TR loss 1.3949 | VA loss 1.4419 | lr 1.00e-03
       TR acc/F1 (sev 0.259 / 0.248) | VA acc/F1 (sev 0.098 / 0.076)
       TR per-class acc (sev [0.110, 0.255, 0.260, 0.410])
       VA per-class acc (sev [0.000, 0.000, 0.667, nan])



[Epoch 004] TR loss 1.3774 | VA loss 1.3375 | lr 1.00e-03
       TR acc/F1 (sev 0.301 / 0.285) | VA acc/F1 (sev 0.317 / 0.235)
       TR per-class acc (sev [0.280, 0.100, 0.340, 0.485])
       VA per-class acc (sev [0.160, 0.900, 0.000, nan])



...

[Epoch 035] TR loss 0.2883 | VA loss 1.4731 | lr 2.50e-04
       TR acc/F1 (sev 0.886 / 0.886) | VA acc/F1 (sev 0.472 / 0.475)
       TR per-class acc (sev [0.940, 0.800, 0.840, 0.965])
       VA per-class acc (sev [1.000, 0.231, 0.533, 0.333])



[Epoch 036] TR loss 0.2439 | VA loss 1.0453 | lr 2.50e-04
       TR acc/F1 (sev 0.910 / 0.910) | VA acc/F1 (sev 0.611 / 0.631)
       TR per-class acc (sev [0.955, 0.830, 0.875, 0.980])
       VA per-class acc (sev [1.000, 0.308, 0.733, 0.667])



[Epoch 037] TR loss 0.2332 | VA loss 1.1394 | lr 2.50e-04
       TR acc/F1 (sev 0.920 / 0.919) | VA acc/F1 (sev 0.556 / 0.533)
       TR per-class acc (sev [0.970, 0.855, 0.870, 0.985])
       VA per-class acc (sev [1.000, 0.308, 0.667, 0.333])



[Epoch 038] TR loss 0.2817 | VA loss 1.1660 | lr 2.50e-04
       TR acc/F1 (sev 0.901 / 0.901) | VA acc/F1 (sev 0.611 / 0.631)
       TR per-class acc (sev [0.955, 0.815, 0.875, 0.960])
       VA per-class acc (sev [1.000, 0.308, 0.733, 0.667])

Early stopping at epoch 38 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8056 | macro-F1=0.7666 | per-class acc=[0.8, 0.615, 1.0, 0.667]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.8780 | macro-F1=0.8637
PID= 2 | ACC=1.0000 | macro-F1=1.0000
PID= 3 | ACC=0.8525 | macro-F1=0.8569
PID= 4 | ACC=0.8387 | macro-F1=0.5644
PID= 5 | ACC=0.9091 | macro-F1=0.6886
PID= 6 | ACC=0.8889 | macro-F1=0.8632
PID= 7 | ACC=1.0000 | macro-F1=1.0000
PID= 8 | ACC=0.9333 | macro-F1=0.9148
PID= 9 | ACC=0.8056 | macro-F1=0.7666
-----------------------------------------------
ACC   mean±std: 0.9007 ± 0.0678
F1    mean±std: 0.8353 ± 0.1424


In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'MyWaveNet'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4526 | VA loss 1.4920 | lr 1.00e-03
       TR acc/F1 (sev 0.266 / 0.261) | VA acc/F1 (sev 0.000 / 0.000)
       TR per-class acc (sev [0.160, 0.335, 0.335, 0.235])
       VA per-class acc (sev [0.000, 0.000, 0.000, nan])



[Epoch 002] TR loss 1.3999 | VA loss 1.4257 | lr 1.00e-03
       TR acc/F1 (sev 0.281 / 0.276) | VA acc/F1 (sev 0.195 / 0.205)
       TR per-class acc (sev [0.190, 0.260, 0.270, 0.405])
       VA per-class acc (sev [0.040, 0.700, 0.000, nan])



[Epoch 003] TR loss 1.3776 | VA loss 1.2496 | lr 1.00e-03
       TR acc/F1 (sev 0.325 / 0.316) | VA acc/F1 (sev 0.220 / 0.120)
       TR per-class acc (sev [0.245, 0.295, 0.225, 0.535])
       VA per-class acc (sev [0.000, 0.900, 0.000, nan])



[Epoch 004] TR loss 1.3020 | VA loss 1.2189 | lr 1.00e-03
       TR acc/F1 (sev 0.333 / 0.319) | VA acc/F1 (sev 0.098 / 0.067)
       TR per-class acc (sev [0.300, 0.135, 0.345, 0.550])
       VA per-class acc (sev [0.000, 0.000, 0.667, nan])



...

[Epoch 050] TR loss 0.1560 | VA loss 1.9943 | lr 1.25e-04
       TR acc/F1 (sev 0.943 / 0.942) | VA acc/F1 (sev 0.556 / 0.559)
       TR per-class acc (sev [0.975, 0.875, 0.935, 0.985])
       VA per-class acc (sev [1.000, 0.231, 0.667, 0.667])



[Epoch 051] TR loss 0.1839 | VA loss 1.4301 | lr 1.25e-04
       TR acc/F1 (sev 0.940 / 0.940) | VA acc/F1 (sev 0.556 / 0.546)
       TR per-class acc (sev [0.965, 0.885, 0.930, 0.980])
       VA per-class acc (sev [1.000, 0.231, 0.600, 1.000])



[Epoch 052] TR loss 0.1884 | VA loss 1.3540 | lr 1.25e-04
       TR acc/F1 (sev 0.936 / 0.936) | VA acc/F1 (sev 0.667 / 0.638)
       TR per-class acc (sev [0.965, 0.860, 0.930, 0.990])
       VA per-class acc (sev [1.000, 0.231, 0.867, 1.000])



[Epoch 053] TR loss 0.1913 | VA loss 1.5284 | lr 1.25e-04
       TR acc/F1 (sev 0.931 / 0.931) | VA acc/F1 (sev 0.611 / 0.589)
       TR per-class acc (sev [0.970, 0.865, 0.910, 0.980])
       VA per-class acc (sev [1.000, 0.231, 0.800, 0.667])

Early stopping at epoch 53 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8333 | macro-F1=0.8505 | per-class acc=[0.8, 0.846, 0.8, 1.0]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.8780 | macro-F1=0.8637
PID= 2 | ACC=0.9600 | macro-F1=0.9035
PID= 3 | ACC=0.7705 | macro-F1=0.6939
PID= 4 | ACC=0.8548 | macro-F1=0.8096
PID= 5 | ACC=0.9091 | macro-F1=0.6886
PID= 6 | ACC=1.0000 | macro-F1=1.0000
PID= 7 | ACC=1.0000 | macro-F1=1.0000
PID= 8 | ACC=0.9333 | macro-F1=0.9175
PID= 9 | ACC=0.8333 | macro-F1=0.8505
-----------------------------------------------
ACC   mean±std: 0.9043 ± 0.0778
F1    mean±std: 0.8586 ± 0.1140


### GAP

In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'LSTM'
time_pool = 'gap'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim,
        time_pool=time_pool,
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{time_pool}_ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4290 | VA loss 1.3947 | lr 1.00e-03
       TR acc/F1 (sev 0.276 / 0.271) | VA acc/F1 (sev 0.146 / 0.085)
       TR per-class acc (sev [0.145, 0.345, 0.280, 0.335])
       VA per-class acc (sev [0.000, 0.000, 1.000, nan])



[Epoch 002] TR loss 1.3936 | VA loss 1.3252 | lr 1.00e-03
       TR acc/F1 (sev 0.286 / 0.279) | VA acc/F1 (sev 0.463 / 0.215)
       TR per-class acc (sev [0.200, 0.290, 0.210, 0.445])
       VA per-class acc (sev [0.760, 0.000, 0.000, nan])



[Epoch 003] TR loss 1.4017 | VA loss 1.4309 | lr 1.00e-03
       TR acc/F1 (sev 0.276 / 0.272) | VA acc/F1 (sev 0.244 / 0.131)
       TR per-class acc (sev [0.185, 0.215, 0.370, 0.335])
       VA per-class acc (sev [0.000, 1.000, 0.000, nan])



[Epoch 004] TR loss 1.3774 | VA loss 1.1490 | lr 1.00e-03
       TR acc/F1 (sev 0.304 / 0.289) | VA acc/F1 (sev 0.610 / 0.253)
       TR per-class acc (sev [0.155, 0.265, 0.260, 0.535])
       VA per-class acc (sev [1.000, 0.000, 0.000, nan])



...

[Epoch 033] TR loss 0.2148 | VA loss 1.7404 | lr 2.50e-04
       TR acc/F1 (sev 0.917 / 0.917) | VA acc/F1 (sev 0.444 / 0.394)
       TR per-class acc (sev [0.950, 0.880, 0.865, 0.975])
       VA per-class acc (sev [1.000, 0.000, 0.667, 0.333])



[Epoch 034] TR loss 0.2080 | VA loss 2.2050 | lr 2.50e-04
       TR acc/F1 (sev 0.910 / 0.909) | VA acc/F1 (sev 0.444 / 0.432)
       TR per-class acc (sev [0.975, 0.805, 0.890, 0.970])
       VA per-class acc (sev [1.000, 0.000, 0.600, 0.667])



[Epoch 035] TR loss 0.1782 | VA loss 2.3377 | lr 2.50e-04
       TR acc/F1 (sev 0.939 / 0.939) | VA acc/F1 (sev 0.472 / 0.484)
       TR per-class acc (sev [0.965, 0.880, 0.940, 0.970])
       VA per-class acc (sev [1.000, 0.000, 0.600, 1.000])



[Epoch 036] TR loss 0.1842 | VA loss 2.4616 | lr 2.50e-04
       TR acc/F1 (sev 0.912 / 0.913) | VA acc/F1 (sev 0.472 / 0.484)
       TR per-class acc (sev [0.965, 0.845, 0.890, 0.950])
       VA per-class acc (sev [1.000, 0.000, 0.600, 1.000])

Early stopping at epoch 36 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8056 | macro-F1=0.7560 | per-class acc=[0.6, 1.0, 0.667, 1.0]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.9024 | macro-F1=0.8834
PID= 2 | ACC=0.9600 | macro-F1=0.9626
PID= 3 | ACC=0.6557 | macro-F1=0.5525
PID= 4 | ACC=0.8548 | macro-F1=0.5778
PID= 5 | ACC=0.9091 | macro-F1=0.8474
PID= 6 | ACC=1.0000 | macro-F1=1.0000
PID= 7 | ACC=0.9167 | macro-F1=0.9111
PID= 8 | ACC=0.9667 | macro-F1=0.9556
PID= 9 | ACC=0.8056 | macro-F1=0.7560
-----------------------------------------------
ACC   mean±std: 0.8857 ± 0.1044
F1    mean±std: 0.8274 ± 0.1651


In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'ResNet18'
time_pool = 'gap'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim,
        time_pool=time_pool,
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{time_pool}_ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4657 | VA loss 1.5031 | lr 1.00e-03
       TR acc/F1 (sev 0.269 / 0.267) | VA acc/F1 (sev 0.146 / 0.085)
       TR per-class acc (sev [0.315, 0.250, 0.205, 0.305])
       VA per-class acc (sev [0.000, 0.000, 1.000, nan])



[Epoch 002] TR loss 1.4023 | VA loss 1.5431 | lr 1.00e-03
       TR acc/F1 (sev 0.259 / 0.255) | VA acc/F1 (sev 0.024 / 0.042)
       TR per-class acc (sev [0.275, 0.195, 0.200, 0.365])
       VA per-class acc (sev [0.000, 0.100, 0.000, nan])



[Epoch 003] TR loss 1.3822 | VA loss 1.3626 | lr 1.00e-03
       TR acc/F1 (sev 0.302 / 0.292) | VA acc/F1 (sev 0.244 / 0.131)
       TR per-class acc (sev [0.205, 0.200, 0.310, 0.495])
       VA per-class acc (sev [0.000, 1.000, 0.000, nan])



[Epoch 004] TR loss 1.3020 | VA loss 1.2636 | lr 1.00e-03
       TR acc/F1 (sev 0.346 / 0.337) | VA acc/F1 (sev 0.195 / 0.119)
       TR per-class acc (sev [0.395, 0.265, 0.195, 0.530])
       VA per-class acc (sev [0.000, 0.800, 0.000, nan])



...

[Epoch 032] TR loss 0.2186 | VA loss 2.0897 | lr 2.50e-04
       TR acc/F1 (sev 0.917 / 0.917) | VA acc/F1 (sev 0.444 / 0.442)
       TR per-class acc (sev [0.970, 0.855, 0.895, 0.950])
       VA per-class acc (sev [1.000, 0.154, 0.400, 1.000])



[Epoch 033] TR loss 0.1877 | VA loss 1.1594 | lr 2.50e-04
       TR acc/F1 (sev 0.934 / 0.933) | VA acc/F1 (sev 0.639 / 0.657)
       TR per-class acc (sev [0.980, 0.880, 0.905, 0.970])
       VA per-class acc (sev [1.000, 0.308, 0.733, 1.000])



[Epoch 034] TR loss 0.1968 | VA loss 1.4586 | lr 2.50e-04
       TR acc/F1 (sev 0.927 / 0.927) | VA acc/F1 (sev 0.528 / 0.517)
       TR per-class acc (sev [0.975, 0.865, 0.905, 0.965])
       VA per-class acc (sev [1.000, 0.308, 0.467, 1.000])



[Epoch 035] TR loss 0.1699 | VA loss 2.1425 | lr 2.50e-04
       TR acc/F1 (sev 0.929 / 0.928) | VA acc/F1 (sev 0.500 / 0.512)
       TR per-class acc (sev [0.980, 0.880, 0.890, 0.965])
       VA per-class acc (sev [1.000, 0.154, 0.533, 1.000])

Early stopping at epoch 35 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8889 | macro-F1=0.8516 | per-class acc=[1.0, 0.769, 1.0, 0.667]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.8537 | macro-F1=0.8144
PID= 2 | ACC=0.3200 | macro-F1=0.2293
PID= 3 | ACC=0.7705 | macro-F1=0.6711
PID= 4 | ACC=0.5484 | macro-F1=0.1771
PID= 5 | ACC=0.8636 | macro-F1=0.6350
PID= 6 | ACC=0.8889 | macro-F1=0.8632
PID= 7 | ACC=1.0000 | macro-F1=1.0000
PID= 8 | ACC=0.9667 | macro-F1=0.9556
PID= 9 | ACC=0.8889 | macro-F1=0.8516
-----------------------------------------------
ACC   mean±std: 0.7890 ± 0.2191
F1    mean±std: 0.6886 ± 0.2994


In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'TimesNet'
time_pool = 'gap'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim,
        time_pool=time_pool,
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{time_pool}_ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

[Fold PID=1]


[Epoch 001] TR loss 1.4412 | VA loss 1.2953 | lr 1.00e-03
       TR acc/F1 (sev 0.266 / 0.265) | VA acc/F1 (sev 0.146 / 0.116)
       TR per-class acc (sev [0.275, 0.195, 0.285, 0.310])
       VA per-class acc (sev [0.160, 0.200, 0.000, nan])



[Epoch 002] TR loss 1.4162 | VA loss 1.3692 | lr 1.00e-03
       TR acc/F1 (sev 0.265 / 0.264) | VA acc/F1 (sev 0.146 / 0.085)
       TR per-class acc (sev [0.270, 0.250, 0.310, 0.230])
       VA per-class acc (sev [0.000, 0.000, 1.000, nan])



[Epoch 003] TR loss 1.3839 | VA loss 1.2665 | lr 1.00e-03
       TR acc/F1 (sev 0.299 / 0.270) | VA acc/F1 (sev 0.415 / 0.199)
       TR per-class acc (sev [0.060, 0.215, 0.405, 0.515])
       VA per-class acc (sev [0.680, 0.000, 0.000, nan])



[Epoch 004] TR loss 1.3848 | VA loss 1.3408 | lr 1.00e-03
       TR acc/F1 (sev 0.284 / 0.272) | VA acc/F1 (sev 0.463 / 0.248)
       TR per-class acc (sev [0.255, 0.110, 0.385, 0.385])
       VA per-class acc (sev [0.760, 0.000, 0.000, nan])



...

[Epoch 043] TR loss 0.3216 | VA loss 1.0834 | lr 1.25e-04
       TR acc/F1 (sev 0.860 / 0.859) | VA acc/F1 (sev 0.556 / 0.527)
       TR per-class acc (sev [0.925, 0.745, 0.815, 0.955])
       VA per-class acc (sev [0.600, 0.923, 0.133, 1.000])



[Epoch 044] TR loss 0.3277 | VA loss 1.0699 | lr 1.25e-04
       TR acc/F1 (sev 0.874 / 0.874) | VA acc/F1 (sev 0.611 / 0.586)
       TR per-class acc (sev [0.935, 0.800, 0.825, 0.935])
       VA per-class acc (sev [0.600, 1.000, 0.200, 1.000])



[Epoch 045] TR loss 0.3096 | VA loss 1.0309 | lr 1.25e-04
       TR acc/F1 (sev 0.869 / 0.869) | VA acc/F1 (sev 0.556 / 0.528)
       TR per-class acc (sev [0.910, 0.800, 0.825, 0.940])
       VA per-class acc (sev [0.600, 0.846, 0.200, 1.000])



[Epoch 046] TR loss 0.2809 | VA loss 0.9229 | lr 1.25e-04
       TR acc/F1 (sev 0.894 / 0.894) | VA acc/F1 (sev 0.639 / 0.613)
       TR per-class acc (sev [0.955, 0.795, 0.875, 0.950])
       VA per-class acc (sev [0.600, 1.000, 0.267, 1.000])

Early stopping at epoch 46 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.8056 | macro-F1=0.7592 | per-class acc=[0.6, 0.846, 0.8, 1.0]

===== LOSO Severity Summary =====
PID= 1 | ACC=0.8780 | macro-F1=0.8637
PID= 2 | ACC=0.9200 | macro-F1=0.8472
PID= 3 | ACC=0.8852 | macro-F1=0.8500
PID= 4 | ACC=0.8226 | macro-F1=0.5601
PID= 5 | ACC=0.9091 | macro-F1=0.6886
PID= 6 | ACC=1.0000 | macro-F1=1.0000
PID= 7 | ACC=0.9167 | macro-F1=0.9161
PID= 8 | ACC=0.9000 | macro-F1=0.8880
PID= 9 | ACC=0.8056 | macro-F1=0.7592
-----------------------------------------------
ACC   mean±std: 0.8930 ± 0.0569
F1    mean±std: 0.8192 ± 0.1315


In [ ]:
acc_name = 'MyWaveNet'
traj_name = 'MyWaveNet'
time_pool = 'gap'

device = DEVICE
epochs = 200
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64

sev_accs, sev_f1s, fold_ids = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    print(f"[Fold PID={patient_id}]")
    set_seed()

    acc_encoder = build_model(
        model_name=acc_name,
        in_ch=3,
        num_classes=4,
    )
    traj_encoder = build_model(
        model_name=traj_name,
        in_ch=2,
        num_classes=3,
    )

    model = MIL_MultiModalTimesNet(
        acc_encoder=acc_encoder,
        traj_encoder=traj_encoder,
        d_model=d_model,
        num_sclass=4,
        ma_heads=8,
        mil_attn_dim=mil_attn_dim,
        time_pool=time_pool,
    ).to(device)

    history = train_model_sev_only(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        epochs=epochs,
        learning_rate=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader(model, dl_va, device, target='severity')
    print(f"[Fold PID={patient_id}] severity ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint3')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{time_pool}_ACC_{acc_name}_Traj_{traj_name}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

acc_mean, acc_std = float(np.mean(sev_accs)), float(np.std(sev_accs, ddof=1))
f1_mean,  f1_std  = float(np.mean(sev_f1s)),  float(np.std(sev_f1s,  ddof=1))

print("\n===== LOSO Severity Summary =====")
for pid, a, f in zip(fold_ids, sev_accs, sev_f1s):
    print(f"PID={pid:>2} | ACC={a:.4f} | macro-F1={f:.4f}")
print("-----------------------------------------------")
print(f"ACC   mean±std: {acc_mean:.4f} ± {acc_std:.4f}")
print(f"F1    mean±std: {f1_mean:.4f} ± {f1_std:.4f}")

## SIngle Modality

### Using ACC Sev CLF

In [ ]:
model_name = 'LSTM'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 1.3859 | VA loss 1.3966 | lr 1.00e-03
       TR acc 0.250 | VA acc 0.024
       TR F1  0.234 | VA F1  0.037
       TR per-class acc [0.430, 0.230, 0.090, 0.250]
       VA per-class acc [0.000, 0.000, 0.167, nan]


[Epoch 002] TR loss 1.3797 | VA loss 1.3672 | lr 1.00e-03
       TR acc 0.282 | VA acc 0.195
       TR F1  0.266 | VA F1  0.121
       TR per-class acc [0.160, 0.140, 0.415, 0.415]
       VA per-class acc [0.320, 0.000, 0.000, nan]


[Epoch 003] TR loss 1.3649 | VA loss 1.3316 | lr 1.00e-03
       TR acc 0.294 | VA acc 0.244
       TR F1  0.250 | VA F1  0.171
       TR per-class acc [0.065, 0.105, 0.385, 0.620]
       VA per-class acc [0.280, 0.300, 0.000, nan]


[Epoch 004] TR loss 1.3643 | VA loss 1.3521 | lr 1.00e-03
       TR acc 0.309 | VA acc 0.341
       TR F1  0.299 | VA F1  0.183
       TR per-class acc [0.230, 0.280, 0.210, 0.515]
       VA per-class acc [0.560, 0.000, 0.000, nan]


[Epoch 005] TR loss 1.3552 | VA loss 1.3433 | lr 1.00e-03
       TR acc 0.318 | VA acc 0.220
       TR F1  0.290 | VA F1  0.156
       TR per-class acc [0.095, 0.195, 0.485, 0.495]
       VA per-class acc [0.240, 0.300, 0.000, nan]


...

[Epoch 036] TR loss 1.2948 | VA loss 1.3689 | lr 1.25e-04
       TR acc 0.380 | VA acc 0.222
       TR F1  0.329 | VA F1  0.183
       TR per-class acc [0.580, 0.005, 0.490, 0.445]
       VA per-class acc [0.400, 0.231, 0.200, 0.000]


[Epoch 037] TR loss 1.2974 | VA loss 1.3365 | lr 1.25e-04
       TR acc 0.378 | VA acc 0.333
       TR F1  0.328 | VA F1  0.258
       TR per-class acc [0.640, 0.015, 0.505, 0.350]
       VA per-class acc [0.400, 0.385, 0.333, 0.000]


[Epoch 038] TR loss 1.3078 | VA loss 1.3032 | lr 1.25e-04
       TR acc 0.361 | VA acc 0.306
       TR F1  0.313 | VA F1  0.199
       TR per-class acc [0.600, 0.015, 0.500, 0.330]
       VA per-class acc [0.200, 0.154, 0.533, 0.000]


[Epoch 039] TR loss 1.2937 | VA loss 1.3038 | lr 6.25e-05
       TR acc 0.370 | VA acc 0.278
       TR F1  0.321 | VA F1  0.153
       TR per-class acc [0.565, 0.010, 0.550, 0.355]
       VA per-class acc [0.200, 0.000, 0.600, 0.000]
Early stopping at epoch 39 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.4167 | macro-F1=0.1471 | per-class acc=[0.0, 0.0, 1.0, 0.0]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.2195       0.2456
         2  0.3600       0.1324
         3  0.4098       0.2649
         4  0.6774       0.3688
         5  0.7273       0.6129
         6  0.6667       0.2667
         7  0.2500       0.3000
         8  0.8333       0.8117
         9  0.4167       0.1471

=== Summary (mean ± std) ===
Accuracy   : 0.5067 ± 0.2229
Macro-F1   : 0.3500 ± 0.2233


In [ ]:
model_name = 'ResNet18'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 2.5202 | VA loss 1.2146 | lr 1.00e-03
       TR acc 0.305 | VA acc 0.537
       TR F1  0.306 | VA F1  0.325
       TR per-class acc [0.360, 0.230, 0.270, 0.360]
       VA per-class acc [0.760, 0.300, 0.000, nan]


[Epoch 002] TR loss 1.4009 | VA loss 1.1632 | lr 1.00e-03
       TR acc 0.356 | VA acc 0.512
       TR F1  0.351 | VA F1  0.277
       TR per-class acc [0.400, 0.240, 0.250, 0.535]
       VA per-class acc [0.800, 0.000, 0.167, nan]


[Epoch 003] TR loss 1.2284 | VA loss 0.9260 | lr 1.00e-03
       TR acc 0.443 | VA acc 0.683
       TR F1  0.442 | VA F1  0.487
       TR per-class acc [0.435, 0.355, 0.350, 0.630]
       VA per-class acc [1.000, 0.000, 0.500, nan]


[Epoch 004] TR loss 1.1543 | VA loss 1.0300 | lr 1.00e-03
       TR acc 0.499 | VA acc 0.537
       TR F1  0.496 | VA F1  0.482
       TR per-class acc [0.530, 0.355, 0.400, 0.710]
       VA per-class acc [0.480, 0.900, 0.167, nan]


[Epoch 005] TR loss 1.1309 | VA loss 2.7132 | lr 1.00e-03
       TR acc 0.519 | VA acc 0.146
       TR F1  0.510 | VA F1  0.129
       TR per-class acc [0.560, 0.340, 0.390, 0.785]
       VA per-class acc [0.240, 0.000, 0.000, nan]


...

[Epoch 033] TR loss 0.4715 | VA loss 1.0051 | lr 1.25e-04
       TR acc 0.806 | VA acc 0.583
       TR F1  0.805 | VA F1  0.442
       TR per-class acc [0.905, 0.685, 0.705, 0.930]
       VA per-class acc [1.000, 0.385, 0.733, 0.000]


[Epoch 034] TR loss 0.4658 | VA loss 2.0671 | lr 1.25e-04
       TR acc 0.821 | VA acc 0.306
       TR F1  0.821 | VA F1  0.248
       TR per-class acc [0.895, 0.710, 0.755, 0.925]
       VA per-class acc [1.000, 0.154, 0.267, 0.000]


[Epoch 035] TR loss 0.4359 | VA loss 2.4567 | lr 1.25e-04
       TR acc 0.833 | VA acc 0.278
       TR F1  0.831 | VA F1  0.230
       TR per-class acc [0.925, 0.725, 0.745, 0.935]
       VA per-class acc [1.000, 0.154, 0.200, 0.000]


[Epoch 036] TR loss 0.4268 | VA loss 2.5578 | lr 6.25e-05
       TR acc 0.815 | VA acc 0.306
       TR F1  0.815 | VA F1  0.252
       TR per-class acc [0.880, 0.740, 0.735, 0.905]
       VA per-class acc [1.000, 0.231, 0.200, 0.000]
Early stopping at epoch 36 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.6944 | macro-F1=0.3897 | per-class acc=[0.0, 0.846, 0.933, 0.0]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.8780       0.8563
         2  0.8800       0.6976
         3  0.6557       0.4191
         4  0.6452       0.2848
         5  0.8182       0.7208
         6  1.0000       1.0000
         7  0.8333       0.8125
         8  0.8333       0.8074
         9  0.6944       0.3897

=== Summary (mean ± std) ===
Accuracy   : 0.8042 ± 0.1177
Macro-F1   : 0.6654 ± 0.2439


In [ ]:
model_name = 'TimesNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 1.4194 | VA loss 1.5625 | lr 1.00e-03
       TR acc 0.268 | VA acc 0.024
       TR F1  0.265 | VA F1  0.039
       TR per-class acc [0.170, 0.255, 0.305, 0.340]
       VA per-class acc [0.000, 0.000, 0.167, nan]


[Epoch 002] TR loss 1.3760 | VA loss 1.4073 | lr 1.00e-03
       TR acc 0.334 | VA acc 0.049
       TR F1  0.325 | VA F1  0.049
       TR per-class acc [0.365, 0.165, 0.455, 0.350]
       VA per-class acc [0.000, 0.200, 0.000, nan]


[Epoch 003] TR loss 1.3612 | VA loss 1.5539 | lr 1.00e-03
       TR acc 0.324 | VA acc 0.098
       TR F1  0.323 | VA F1  0.089
       TR per-class acc [0.300, 0.275, 0.390, 0.330]
       VA per-class acc [0.000, 0.000, 0.667, nan]


[Epoch 004] TR loss 1.3665 | VA loss 1.3795 | lr 1.00e-03
       TR acc 0.331 | VA acc 0.122
       TR F1  0.324 | VA F1  0.101
       TR per-class acc [0.375, 0.220, 0.245, 0.485]
       VA per-class acc [0.000, 0.500, 0.000, nan]


[Epoch 005] TR loss 1.3679 | VA loss 1.3274 | lr 1.00e-03
       TR acc 0.343 | VA acc 0.146
       TR F1  0.334 | VA F1  0.140
       TR per-class acc [0.295, 0.200, 0.500, 0.375]
       VA per-class acc [0.080, 0.000, 0.667, nan]


...

[Epoch 028] TR loss 1.2168 | VA loss 1.2624 | lr 1.25e-04
       TR acc 0.471 | VA acc 0.333
       TR F1  0.452 | VA F1  0.196
       TR per-class acc [0.550, 0.165, 0.505, 0.665]
       VA per-class acc [0.200, 0.077, 0.667, 0.000]


[Epoch 029] TR loss 1.2269 | VA loss 1.2725 | lr 1.25e-04
       TR acc 0.410 | VA acc 0.333
       TR F1  0.394 | VA F1  0.213
       TR per-class acc [0.445, 0.130, 0.440, 0.625]
       VA per-class acc [0.200, 0.154, 0.600, 0.000]


[Epoch 030] TR loss 1.2044 | VA loss 1.2631 | lr 1.25e-04
       TR acc 0.443 | VA acc 0.306
       TR F1  0.413 | VA F1  0.200
       TR per-class acc [0.520, 0.095, 0.470, 0.685]
       VA per-class acc [0.200, 0.154, 0.533, 0.000]


[Epoch 031] TR loss 1.1590 | VA loss 1.2767 | lr 6.25e-05
       TR acc 0.485 | VA acc 0.333
       TR F1  0.460 | VA F1  0.237
       TR per-class acc [0.525, 0.155, 0.545, 0.715]
       VA per-class acc [0.200, 0.308, 0.467, 0.000]
Early stopping at epoch 31 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.2778 | macro-F1=0.1087 | per-class acc=[0.0, 0.769, 0.0, 0.0]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.8780       0.8374
         2  0.3200       0.1754
         3  0.3934       0.2056
         4  0.4677       0.3208
         5  0.8182       0.8172
         6  0.6667       0.2667
         7  0.5000       0.4375
         8  0.5000       0.4223
         9  0.2778       0.1087

=== Summary (mean ± std) ===
Accuracy   : 0.5358 ± 0.2105
Macro-F1   : 0.3991 ± 0.2656


In [ ]:
model_name = 'MyWaveNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 1.5077 | VA loss 1.3132 | lr 1.00e-03
       TR acc 0.291 | VA acc 0.293
       TR F1  0.291 | VA F1  0.185
       TR per-class acc [0.245, 0.290, 0.295, 0.335]
       VA per-class acc [0.080, 1.000, 0.000, nan]


[Epoch 002] TR loss 1.4063 | VA loss 1.4534 | lr 1.00e-03
       TR acc 0.255 | VA acc 0.244
       TR F1  0.253 | VA F1  0.186
       TR per-class acc [0.250, 0.295, 0.300, 0.175]
       VA per-class acc [0.240, 0.000, 0.667, nan]


[Epoch 003] TR loss 1.3947 | VA loss 1.4422 | lr 1.00e-03
       TR acc 0.306 | VA acc 0.073
       TR F1  0.302 | VA F1  0.061
       TR per-class acc [0.235, 0.230, 0.410, 0.350]
       VA per-class acc [0.000, 0.300, 0.000, nan]


[Epoch 004] TR loss 1.3629 | VA loss 1.5070 | lr 1.00e-03
       TR acc 0.311 | VA acc 0.049
       TR F1  0.301 | VA F1  0.044
       TR per-class acc [0.195, 0.185, 0.445, 0.420]
       VA per-class acc [0.000, 0.200, 0.000, nan]


[Epoch 005] TR loss 1.3151 | VA loss 1.7094 | lr 1.00e-03
       TR acc 0.375 | VA acc 0.244
       TR F1  0.358 | VA F1  0.189
       TR per-class acc [0.390, 0.125, 0.395, 0.590]
       VA per-class acc [0.000, 0.900, 0.167, nan]


...

[Epoch 027] TR loss 0.2792 | VA loss 2.3342 | lr 2.50e-04
       TR acc 0.895 | VA acc 0.306
       TR F1  0.895 | VA F1  0.243
       TR per-class acc [0.965, 0.830, 0.850, 0.935]
       VA per-class acc [1.000, 0.077, 0.333, 0.000]


[Epoch 028] TR loss 0.2724 | VA loss 4.5216 | lr 2.50e-04
       TR acc 0.882 | VA acc 0.194
       TR F1  0.882 | VA F1  0.157
       TR per-class acc [0.955, 0.820, 0.805, 0.950]
       VA per-class acc [1.000, 0.000, 0.133, 0.000]


[Epoch 029] TR loss 0.2686 | VA loss 3.1744 | lr 2.50e-04
       TR acc 0.890 | VA acc 0.222
       TR F1  0.890 | VA F1  0.178
       TR per-class acc [0.970, 0.810, 0.825, 0.955]
       VA per-class acc [1.000, 0.077, 0.133, 0.000]


[Epoch 030] TR loss 0.2740 | VA loss 4.7101 | lr 1.25e-04
       TR acc 0.896 | VA acc 0.194
       TR F1  0.897 | VA F1  0.157
       TR per-class acc [0.975, 0.840, 0.850, 0.920]
       VA per-class acc [1.000, 0.000, 0.133, 0.000]
Early stopping at epoch 30 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.7778 | macro-F1=0.7606 | per-class acc=[1.0, 0.462, 0.933, 1.0]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.8293       0.7582
         2  0.9600       0.9035
         3  0.6066       0.5181
         4  0.7097       0.7048
         5  0.9091       0.9114
         6  0.8889       0.8632
         7  1.0000       1.0000
         8  0.9333       0.9068
         9  0.7778       0.7606

=== Summary (mean ± std) ===
Accuracy   : 0.8461 ± 0.1280
Macro-F1   : 0.8141 ± 0.1452


### Using Traj Sev CLF

In [ ]:
model_name = 'LSTM'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 1.3904 | VA loss 1.3812 | lr 1.00e-03
       TR acc 0.237 | VA acc 0.244
       TR F1  0.121 | VA F1  0.131
       TR per-class acc [0.000, 0.020, 0.890, 0.040]
       VA per-class acc [0.000, 1.000, 0.000, nan]


[Epoch 002] TR loss 1.3874 | VA loss 1.3819 | lr 1.00e-03
       TR acc 0.240 | VA acc 0.537
       TR F1  0.217 | VA F1  0.249
       TR per-class acc [0.150, 0.415, 0.335, 0.060]
       VA per-class acc [0.880, 0.000, 0.000, nan]


[Epoch 003] TR loss 1.3877 | VA loss 1.3817 | lr 1.00e-03
       TR acc 0.234 | VA acc 0.463
       TR F1  0.150 | VA F1  0.325
       TR per-class acc [0.150, 0.750, 0.015, 0.020]
       VA per-class acc [0.600, 0.000, 0.667, nan]


[Epoch 004] TR loss 1.3798 | VA loss 1.4354 | lr 1.00e-03
       TR acc 0.291 | VA acc 0.122
       TR F1  0.276 | VA F1  0.148
       TR per-class acc [0.400, 0.190, 0.435, 0.140]
       VA per-class acc [0.000, 0.200, 0.500, nan]


[Epoch 005] TR loss 1.3857 | VA loss 1.3816 | lr 1.00e-03
       TR acc 0.256 | VA acc 0.244
       TR F1  0.237 | VA F1  0.203
       TR per-class acc [0.395, 0.085, 0.390, 0.155]
       VA per-class acc [0.160, 0.000, 1.000, nan]


...

[Epoch 042] TR loss 1.0090 | VA loss 1.2220 | lr 1.25e-04
       TR acc 0.517 | VA acc 0.417
       TR F1  0.504 | VA F1  0.287
       TR per-class acc [0.595, 0.200, 0.475, 0.800]
       VA per-class acc [0.600, 0.077, 0.733, 0.000]


[Epoch 043] TR loss 1.0177 | VA loss 1.2581 | lr 1.25e-04
       TR acc 0.521 | VA acc 0.389
       TR F1  0.510 | VA F1  0.361
       TR per-class acc [0.570, 0.265, 0.430, 0.820]
       VA per-class acc [0.600, 0.154, 0.467, 0.667]


[Epoch 044] TR loss 0.9904 | VA loss 1.2399 | lr 1.25e-04
       TR acc 0.512 | VA acc 0.389
       TR F1  0.505 | VA F1  0.330
       TR per-class acc [0.555, 0.305, 0.345, 0.845]
       VA per-class acc [0.800, 0.077, 0.533, 0.333]


[Epoch 045] TR loss 1.0033 | VA loss 1.2599 | lr 6.25e-05
       TR acc 0.519 | VA acc 0.361
       TR F1  0.506 | VA F1  0.331
       TR per-class acc [0.580, 0.270, 0.380, 0.845]
       VA per-class acc [0.600, 0.077, 0.467, 0.667]
Early stopping at epoch 45 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.4167 | macro-F1=0.2932 | per-class acc=[0.2, 0.308, 0.667, 0.0]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.6829       0.4722
         2  0.3600       0.1324
         3  0.2623       0.2117
         4  0.5806       0.1856
         5  0.5000       0.2778
         6  0.4444       0.3175
         7  0.4167       0.3571
         8  0.2667       0.2105
         9  0.4167       0.2932

=== Summary (mean ± std) ===
Accuracy   : 0.4367 ± 0.1376
Macro-F1   : 0.2731 ± 0.1026


In [ ]:
model_name = 'ResNet18'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 1.5255 | VA loss 0.9923 | lr 1.00e-03
       TR acc 0.352 | VA acc 0.659
       TR F1  0.348 | VA F1  0.450
       TR per-class acc [0.405, 0.220, 0.305, 0.480]
       VA per-class acc [0.880, 0.000, 0.833, nan]


[Epoch 002] TR loss 1.3903 | VA loss 0.9067 | lr 1.00e-03
       TR acc 0.414 | VA acc 0.610
       TR F1  0.401 | VA F1  0.393
       TR per-class acc [0.425, 0.250, 0.300, 0.680]
       VA per-class acc [0.880, 0.000, 0.500, nan]


[Epoch 003] TR loss 1.1623 | VA loss 1.0206 | lr 1.00e-03
       TR acc 0.432 | VA acc 0.293
       TR F1  0.424 | VA F1  0.312
       TR per-class acc [0.505, 0.270, 0.250, 0.705]
       VA per-class acc [0.200, 0.100, 1.000, nan]


[Epoch 004] TR loss 1.1483 | VA loss 1.3336 | lr 1.00e-03
       TR acc 0.444 | VA acc 0.268
       TR F1  0.437 | VA F1  0.262
       TR per-class acc [0.380, 0.215, 0.455, 0.725]
       VA per-class acc [0.320, 0.000, 0.500, nan]


[Epoch 005] TR loss 1.1338 | VA loss 1.1107 | lr 1.00e-03
       TR acc 0.482 | VA acc 0.488
       TR F1  0.458 | VA F1  0.412
       TR per-class acc [0.650, 0.210, 0.290, 0.780]
       VA per-class acc [0.560, 0.100, 0.833, nan]


...

[Epoch 021] TR loss 0.9001 | VA loss 1.1307 | lr 2.50e-04
       TR acc 0.583 | VA acc 0.444
       TR F1  0.568 | VA F1  0.472
       TR per-class acc [0.735, 0.265, 0.470, 0.860]
       VA per-class acc [0.800, 0.077, 0.600, 0.667]


[Epoch 022] TR loss 0.8642 | VA loss 1.1552 | lr 2.50e-04
       TR acc 0.601 | VA acc 0.333
       TR F1  0.594 | VA F1  0.335
       TR per-class acc [0.750, 0.385, 0.410, 0.860]
       VA per-class acc [1.000, 0.077, 0.333, 0.333]


[Epoch 023] TR loss 0.8488 | VA loss 1.1625 | lr 2.50e-04
       TR acc 0.609 | VA acc 0.417
       TR F1  0.603 | VA F1  0.433
       TR per-class acc [0.740, 0.340, 0.520, 0.835]
       VA per-class acc [1.000, 0.077, 0.467, 0.667]


[Epoch 024] TR loss 0.8716 | VA loss 1.2330 | lr 1.25e-04
       TR acc 0.586 | VA acc 0.361
       TR F1  0.583 | VA F1  0.396
       TR per-class acc [0.675, 0.345, 0.470, 0.855]
       VA per-class acc [0.800, 0.154, 0.333, 0.667]
Early stopping at epoch 24 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.4444 | macro-F1=0.3295 | per-class acc=[0.4, 0.385, 0.6, 0.0]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.6585       0.4470
         2  0.2400       0.1399
         3  0.4426       0.3146
         4  0.5323       0.1774
         5  0.7727       0.8111
         6  0.5556       0.2778
         7  0.4167       0.4444
         8  0.1333       0.2109
         9  0.4444       0.3295

=== Summary (mean ± std) ===
Accuracy   : 0.4662 ± 0.1965
Macro-F1   : 0.3503 ± 0.2034


In [ ]:
model_name = 'TimesNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

[Epoch 001] TR loss 1.3720 | VA loss 1.0192 | lr 1.00e-03
       TR acc 0.311 | VA acc 0.610
       TR F1  0.311 | VA F1  0.330
       TR per-class acc [0.285, 0.310, 0.285, 0.365]
       VA per-class acc [0.960, 0.000, 0.167, nan]


[Epoch 002] TR loss 1.3398 | VA loss 1.2421 | lr 1.00e-03
       TR acc 0.360 | VA acc 0.488
       TR F1  0.358 | VA F1  0.445
       TR per-class acc [0.420, 0.240, 0.385, 0.395]
       VA per-class acc [0.680, 0.000, 0.500, nan]


[Epoch 003] TR loss 1.2859 | VA loss 1.3266 | lr 1.00e-03
       TR acc 0.369 | VA acc 0.415
       TR F1  0.366 | VA F1  0.307
       TR per-class acc [0.325, 0.260, 0.410, 0.480]
       VA per-class acc [0.600, 0.000, 0.333, nan]


[Epoch 004] TR loss 1.2970 | VA loss 1.1561 | lr 1.00e-03
       TR acc 0.389 | VA acc 0.585
       TR F1  0.382 | VA F1  0.401
       TR per-class acc [0.470, 0.215, 0.410, 0.460]
       VA per-class acc [0.880, 0.000, 0.333, nan]


[Epoch 005] TR loss 1.2863 | VA loss 1.3738 | lr 1.00e-03
       TR acc 0.376 | VA acc 0.439
       TR F1  0.371 | VA F1  0.324
       TR per-class acc [0.330, 0.250, 0.390, 0.535]
       VA per-class acc [0.680, 0.000, 0.167, nan]


...

[Epoch 022] TR loss 1.0829 | VA loss 1.2355 | lr 2.50e-04
       TR acc 0.532 | VA acc 0.361
       TR F1  0.501 | VA F1  0.224
       TR per-class acc [0.675, 0.150, 0.485, 0.820]
       VA per-class acc [0.400, 0.000, 0.733, 0.000]


[Epoch 023] TR loss 1.0652 | VA loss 1.1842 | lr 2.50e-04
       TR acc 0.542 | VA acc 0.333
       TR F1  0.521 | VA F1  0.280
       TR per-class acc [0.675, 0.215, 0.530, 0.750]
       VA per-class acc [0.400, 0.077, 0.533, 0.333]


[Epoch 024] TR loss 1.0499 | VA loss 1.2587 | lr 2.50e-04
       TR acc 0.541 | VA acc 0.306
       TR F1  0.521 | VA F1  0.265
       TR per-class acc [0.645, 0.230, 0.500, 0.790]
       VA per-class acc [0.400, 0.077, 0.467, 0.333]


[Epoch 025] TR loss 1.0558 | VA loss 1.2197 | lr 1.25e-04
       TR acc 0.517 | VA acc 0.306
       TR F1  0.493 | VA F1  0.309
       TR per-class acc [0.610, 0.165, 0.545, 0.750]
       VA per-class acc [0.400, 0.231, 0.267, 0.667]
Early stopping at epoch 25 (no improvement for 20 epochs).


[Fold PID=9] severity ACC=0.5000 | macro-F1=0.4614 | per-class acc=[0.6, 0.308, 0.6, 0.667]
=== Per-patient severity results ===
patient_id sev_acc sev_macro_f1
         1  0.6098       0.3302
         2  0.3200       0.2242
         3  0.3443       0.2308
         4  0.3226       0.1600
         5  0.5455       0.3431
         6  0.6667       0.2667
         7  0.3333       0.2857
         8  0.3333       0.2941
         9  0.5000       0.4614

=== Summary (mean ± std) ===
Accuracy   : 0.4417 ± 0.1392
Macro-F1   : 0.2885 ± 0.0860


In [ ]:
model_name = 'MyWaveNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='severity'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

### Using ACC Task CLF

In [ ]:
model_name = 'LSTM'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
model_name = 'ResNet18'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
model_name = 'TimesNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
model_name = 'MyWaveNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'acc'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

### Using Traj Task CLF

In [ ]:
model_name = 'LSTM'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
model_name = 'ResNet18'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
model_name = 'TimesNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")

In [ ]:
model_name = 'MyWaveNet'
device = DEVICE
epochs = 100
lr     = 1e-3
patience = 20
d_model = 128
mil_attn_dim = 64
modality = 'traj'
target='task'
num_classes = 4 if target == 'severity' else 3
in_ch = 3 if modality == 'acc' else 2

fold_ids, sev_accs, sev_f1s = [], [], []

for patient_id, (dl_tr, dl_va) in loaders.items():
    set_seed()

    base_encoder = build_model(
        model_name=model_name,
        in_ch=in_ch,
        num_classes=num_classes,
    )
    model = MIL_Single(
        base_encoder=base_encoder,
        num_classes=num_classes,
        d_model=d_model,
        time_dropout=0.1,
        attn_dim=mil_attn_dim,
        mil_dropout=0.1,
    ).to(device)

    history = train_model_single(
        model,
        train_loader=dl_tr,
        valid_loader=dl_va,
        device=device,
        modality=modality,
        target=target,
        epochs=epochs,
        lr=lr,
        early_stopping_patience=patience,
        weight_decay=1e-4,
        grad_clip=1.0,
        sched_factor=0.5,
        sched_patience=5,
    )

    metrics = evaluate_on_loader_single(
        model,
        loader=dl_va,
        device=device,
        target=target,
        modality=modality,
    )
    print(f"[Fold PID={patient_id}] {target} ACC={metrics['acc']:.4f} | macro-F1={metrics['macro_f1']:.4f} "
          f"| per-class acc={np.round(metrics['per_class_acc'], 3).tolist()}")

    base = Path(f'/content/drive/MyDrive/MS/ETAnalysis/checkpoint4')
    os.makedirs(base, exist_ok=True)
    model_path = base / f'{model_name}_{modality}_{target}_val_pid{patient_id}.pth'
    torch.save(model.state_dict(), model_path)

    fold_ids.append(patient_id)
    sev_accs.append(metrics["acc"])
    sev_f1s.append(metrics["macro_f1"])

# fold_ids, sev_accs, sev_f1s 가 이미 채워져 있다고 가정
res_df = pd.DataFrame({
    "patient_id": fold_ids,
    "sev_acc": sev_accs,
    "sev_macro_f1": sev_f1s
}).sort_values("patient_id").reset_index(drop=True)

# 환자별 결과 출력
print("=== Per-patient severity results ===")
print(res_df.to_string(index=False, formatters={
    "sev_acc":     lambda x: f"{x:.4f}",
    "sev_macro_f1":lambda x: f"{x:.4f}",
}))

# 평균 / 표준편차 (표본 표준편차 ddof=1)
acc_mean = float(np.mean(res_df["sev_acc"]))
acc_std  = float(np.std(res_df["sev_acc"], ddof=1))
f1_mean  = float(np.mean(res_df["sev_macro_f1"]))
f1_std   = float(np.std(res_df["sev_macro_f1"], ddof=1))

print("\n=== Summary (mean ± std) ===")
print(f"Accuracy   : {acc_mean:.4f} ± {acc_std:.4f}")
print(f"Macro-F1   : {f1_mean:.4f} ± {f1_std:.4f}")